In [2381]:
import yfinance as yf
import pandas as pd
import talib as ta
import numpy as np
import math
import matplotlib.pyplot as plt
from copy import deepcopy

# Variable

In [2382]:
# variable_data ประวัติการซื้อขายแต่ละกลยุทธ์
column_data_types = {
                    "date": object,
                    "time": object,
                    "type": object,
                    "avg_price": float ,
                    "sum_volume": int,
                    "increase_volume": int,
                    "net_volume": int,
                    "price_sell": float,
                    "volume_sell": int,
                    "price_buy": float,
                    "volume_buy": int,
                    "point_buyback": float,
                    "cash_sell": float,
                    "cash_buy": float,
                    "cash": float,
                    "cashflow_back": float,
                    "cashflow_mid": float,
                    "cashflow_front": float,
                    "cashflow_convert": float,
                    "cashflow_short": float,
                    "cashflow_buymore": float,
                    }
trade_hist = pd.DataFrame(columns=column_data_types.keys()).astype(column_data_types)

# variable_data ปะวัติการซื้อขายทุกวัน
column_data2_types = {
                        "date": object ,
                        "sum_volume": int,
                        "increase_volume": int,
                        "net_volume": int,
                        "cash_buyback": float,
                        "cashflow_buymore": float,
                        "cashflow_reserve": float,
                        "cashflow_fee": float,
                        "total_cash": float,
                        "total_equity": float,
                        "nav": float,
                        "benchmark": float                    
                    }
df_dt = pd.DataFrame(columns=column_data2_types.keys()).astype(column_data2_types)

# variable_data ตัวแปรกลยุทธ์ต่างๆ
dict_variable = {
                'FIRST_PRICE': 0,
                'FIRST_VOLUME': 0, 
                'start_in_tradezone': False,
                'count_index': 0,
                'sum_volume': 0 ,
                'data_index': 0,
                'increase_volume': 0,
                'avg_price': 0,
                'net_volume_percentage': 0,
                'net_volume': 0,
                'date': object,
                'benchmark': 0,
                'total_equity': 0,
                'cashflow_buymore': 0,
                'lower_price': 0,
                'current_price': 0,
                'budget': 0,
                
                # กองหลัง
                'start_in_backzone': False,
                'signal_backprice': 0,
                'list_range_backprice': [],
                'count_range_backprice': 0,
                'list_data_index_back': [],
                'list_sell_backprice': [],
                'list_sell_backvolume': [],
                'list_history_sell_backprice': [],
                'count_back': 0,
                'stat_count_back': 0,
                'cashflow_back': 0,
                'previous_backvolume': 0,

                # กองกลาง
                'list_sell_midprice': [],
                'list_sell_midvolume': [],
                'count_fibo': 0,
                'stat_count_mid': 0,
                'list_level_pricefibo': [],
                'list_level_volumefibo': [],
                'list_avgcost_pricefibo': [],
                'list_data_index_mid': [],
                'signal_buyback_midband': False,
                'cashflow_mid': 0,
                'net_sell_midvolume': 0,
                'list_buy_midcashflow': [],
                'list_sell_midcashflow': [],
                'list_dif_midcashflow': [],

                # กองหน้า
                'list_sell_frontvolume':[],
                'list_sell_frontprice':[],
                'list_level_volumefront': [],
                'count_front': 0,
                'list_data_index_front': [],
                'buyback_pricefront': 0,
                'cashflow_front': 0,

                # ซื้อเพิ่ม
                'start_in_buyincrease': False,

                # แปลงร่างกองหลัง
                'cashflow_convert': 0,
                'volume_convert': 0,
                'start_send_convert': False,
                'start_recieve_convert': False,
                'selected_price': 0,
                'selected_convert': str,
                'selected_cashflow': 0,
                'list_data_index_convert': [],

                # แปลงร่างกองหน้า
                'start_in_frontconvert': False,
                'selected_frontprice': 0,
                'selected_frontvolume': 0,
                'selected_frontcashflow': 0,
                'selected_frontconvert': str,
                'list_data_index_frontconvert': [],

                # แปลงร่าง sl
                'count_sl_convert': 0,
                'count_slzone': 0,
                'selected_slprice': 0,
                'selected_slvolume': 0,
                'selected_slcashflow': 0,
                'selected_slconvert': str,
                'list_data_index_sl': [],
                'lower_price_convert': 0,
                'list_namestock_slconvert': [],
                'volume_slconvert': 0,
                'cashflow_slconvert': 0,


                # short
                'start_in_shortzone': False,
                'list_sell_shortprice': [],
                'list_sell_shortvolume': [],
                'list_data_index_short': [],
                'cashflow_short': 0,
                'count_short': 0,
                'list_buy_shortprice': [],
                'signal_shortprice': 0,

                # stat
                'test_max': 0,
                'test_min': 999,
                
                

                }

# dict เก็บเงินรวมทั้งพอร์ต
dict_all_sum_port = {'cashflow_buymore': 0,
                    'cashflow_reserve': 0,
                    'cashflow_fee': 0,
                    'sum_cashflow_back': 0,
                    'sum_cashflow_mid': 0,
                    'sum_cashflow_front': 0,
                    'sum_cashflow_convert': 0,
                    'sum_cashflow_short': 0,
                    }

# dict เก็บ data แยกหุ้นแต่ละตัว
df_trade_hist = {} 
var = {}  
df = {}

# Function

## calculate_spread

In [2383]:
def calculate_spread(price):
    spread = 0
    # หา spread แต่ละระดับราคา
    if price < 2:
      spread = 0.01
    elif price < 5:
      spread = 0.02
    elif price < 10:
      spread = 0.05
    elif price < 25:
      spread = 0.10
    elif price < 100:
      spread = 0.25
    elif price < 200:
      spread = 0.50
    elif price < 400 :
      spread = 1
    elif price >= 400:
      spread = 2
    return spread

def test_calculate_spread(price, expected_output):
    actual_output = calculate_spread(price)
    if actual_output == expected_output:
        print('[PASS]: price {}, expect: {}, actual: {}'.format(price, expected_output, actual_output))
    else:
        print('[FAILED]: price {}, expect: {}, actual: {}'.format(price, expected_output, actual_output))      

test_data = [
    [1.99, 0.01],
    [2.00, 0.02],
    [2.98, 0.02],
    [5.00, 0.05],
    [9.95, 0.05],
    [10.00, 0.10],
    [24.90, 0.10],
    [25.00, 0.25],
    [99.75, 0.25],
    [100.00, 0.50],
    [199.00, 0.50],
    [200.00, 1.00],
    [399.00, 1.00],
    [400.00, 2.00],  
]

for i in range(0,len(test_data),1):  
    test_calculate_spread(test_data[i][0],test_data[i][1] )

[PASS]: price 1.99, expect: 0.01, actual: 0.01
[PASS]: price 2.0, expect: 0.02, actual: 0.02
[PASS]: price 2.98, expect: 0.02, actual: 0.02
[PASS]: price 5.0, expect: 0.05, actual: 0.05
[PASS]: price 9.95, expect: 0.05, actual: 0.05
[PASS]: price 10.0, expect: 0.1, actual: 0.1
[PASS]: price 24.9, expect: 0.1, actual: 0.1
[PASS]: price 25.0, expect: 0.25, actual: 0.25
[PASS]: price 99.75, expect: 0.25, actual: 0.25
[PASS]: price 100.0, expect: 0.5, actual: 0.5
[PASS]: price 199.0, expect: 0.5, actual: 0.5
[PASS]: price 200.0, expect: 1.0, actual: 1
[PASS]: price 399.0, expect: 1.0, actual: 1
[PASS]: price 400.0, expect: 2.0, actual: 2


## round_spread

In [2384]:
def round_spread(price):
  #ปัดราคาหุ้น spread ลงมาที่ใกล้ที่สุด
  if price < 400:
    if (((round((price % 1)*100)) % (calculate_spread(price) * 100)) != 0  or round(price % 1 ,2) > 0.99):
      find_int1 = price // 1
      find_flot1 = price % 1
      find_spread1 = calculate_spread(price)
      find_round1 = ((find_flot1/find_spread1)//1)
      price = find_int1 + (find_spread1*find_round1)
      price = round(price, 2)
  else:
    price = price - price % calculate_spread(price)
  price = round(price,2)
  return price

def test_round_spread(price,expected_output):
    actual_output = round_spread(price)
    if actual_output == expected_output:
        print('[PASS]: price {}, expect: {}, actual: {}'.format(price, expected_output, actual_output))
    else:
        print('[FAILED]: price {}, expect: {}, actual: {}'.format(price, expected_output, actual_output))

test_data = [
    [2.00,2.00],
    [5.00,5.00],
    [10.00,10.00],
    [25.00,25.00],
    [100.00,100.00],
    [200.00,200.00],
    [400.00,400.00],
    [2.01,2.00],
    [5.01,5.00],
    [10.01,10.00],
    [25.01,25.00],
    [100.01,100.00],
    [200.01,200.00],
    [400.01,400.00],
    [3.01,3.00],
    [6.01,6.00],
    [11.01,11.00],
    [26.01,26.00],
    [101.01,101.00],
    [201.01,201.00],
    [401.01,400.00],
    [1.9999,1.99],
    [4.9999,4.98],
    [9.9999,9.95],
    [24.9999,24.90],
    [99.9999,99.75],
    [199.9999,199.50],
    [399.9999,399.00],
]

for i in range(0,len(test_data),1):  
    test_round_spread(test_data[i][0],test_data[i][1] )

[PASS]: price 2.0, expect: 2.0, actual: 2.0
[PASS]: price 5.0, expect: 5.0, actual: 5.0
[PASS]: price 10.0, expect: 10.0, actual: 10.0
[PASS]: price 25.0, expect: 25.0, actual: 25.0
[PASS]: price 100.0, expect: 100.0, actual: 100.0
[PASS]: price 200.0, expect: 200.0, actual: 200.0
[PASS]: price 400.0, expect: 400.0, actual: 400.0
[PASS]: price 2.01, expect: 2.0, actual: 2.0
[PASS]: price 5.01, expect: 5.0, actual: 5.0
[PASS]: price 10.01, expect: 10.0, actual: 10.0
[PASS]: price 25.01, expect: 25.0, actual: 25.0
[PASS]: price 100.01, expect: 100.0, actual: 100.0
[PASS]: price 200.01, expect: 200.0, actual: 200.0
[PASS]: price 400.01, expect: 400.0, actual: 400.0
[PASS]: price 3.01, expect: 3.0, actual: 3.0
[PASS]: price 6.01, expect: 6.0, actual: 6.0
[PASS]: price 11.01, expect: 11.0, actual: 11.0
[PASS]: price 26.01, expect: 26.0, actual: 26.0
[PASS]: price 101.01, expect: 101.0, actual: 101.0
[PASS]: price 201.01, expect: 201.0, actual: 201.0
[PASS]: price 401.01, expect: 400.0, actu

## calculate_floor_stock

In [2385]:
def calculate_floor_stock(volume):
    if volume > 0:
        result = math.floor(abs( volume ) / 100 ) * 100
    else:
        result = 0
    return result

def test_calculate_floor_stock(volume , expected_output):
    actual_output = calculate_floor_stock(volume)
    
    if actual_output == expected_output:
        print('[PASS]: price {}, expect: {}, actual: {}'.format(volume, expected_output, actual_output))
    else:
        print('[FAILED]: price {}, expect: {}, actual: {}'.format(volume, expected_output, actual_output)) 

test_data = [
    [100.00,100.00],
    [200.00,200.00],
    [400.00,400.00],
    [2.01,0],
    [5.01,0],
    [10.01,0],
    [25.01,0],
    [100.01,100.00],
    [25685.01,25600.00],
    [456065.01,456000.00],
    [3.01,0],
    [6.01,0],
    [11.01,0],
    [26.01,0],
    [101.01,100],
    [201.01,200],
    [401.01,400],
    [1.9999,0],
    [4.9999,0],
    [199.9999,100],
    [399.9999,300],
    [-0.9999,0],
    [-100.,0],
    [-399.9999,0],
]

for i in range(0,len(test_data),1):  
    test_calculate_floor_stock(test_data[i][0],test_data[i][1] )  

[PASS]: price 100.0, expect: 100.0, actual: 100
[PASS]: price 200.0, expect: 200.0, actual: 200
[PASS]: price 400.0, expect: 400.0, actual: 400
[PASS]: price 2.01, expect: 0, actual: 0
[PASS]: price 5.01, expect: 0, actual: 0
[PASS]: price 10.01, expect: 0, actual: 0
[PASS]: price 25.01, expect: 0, actual: 0
[PASS]: price 100.01, expect: 100.0, actual: 100
[PASS]: price 25685.01, expect: 25600.0, actual: 25600
[PASS]: price 456065.01, expect: 456000.0, actual: 456000
[PASS]: price 3.01, expect: 0, actual: 0
[PASS]: price 6.01, expect: 0, actual: 0
[PASS]: price 11.01, expect: 0, actual: 0
[PASS]: price 26.01, expect: 0, actual: 0
[PASS]: price 101.01, expect: 100, actual: 100
[PASS]: price 201.01, expect: 200, actual: 200
[PASS]: price 401.01, expect: 400, actual: 400
[PASS]: price 1.9999, expect: 0, actual: 0
[PASS]: price 4.9999, expect: 0, actual: 0
[PASS]: price 199.9999, expect: 100, actual: 100
[PASS]: price 399.9999, expect: 300, actual: 300
[PASS]: price -0.9999, expect: 0, act

## calculate_buyback_point

In [2386]:
def calculate_buyback_point(price,down_range_spread,type_choose):
    # type_choose = -1 คือหาฝั่งลดลง / type_choose = 1 คือหาฝั่งเพิ่ม
    count = 0
    down_price = 0
    price = round_spread(price) 
    if type_choose == -1:
        while count < down_range_spread :
            if price in [2, 5, 10, 25, 100, 200, 400]:
                down_price =  round_spread(price - 0.01)
                price = down_price
                count += 1
                continue
            down_price = round_spread(price - calculate_spread(price))
            price = down_price
            count += 1
        return round_spread(down_price)
    
    elif type_choose == 1:
        while count < down_range_spread :
            down_price = price + calculate_spread(price)
            price = down_price
            count += 1
        return round_spread(down_price)

def test_calculate_buyback_point(price,down_range_spread,type_choose,expected_output):
    actual_output = calculate_buyback_point(price,down_range_spread,type_choose)
    if actual_output == expected_output:
        print('[PASS]: price {},range {},type_choose {}, expect: {}, actual: {}'.format(price, down_range_spread,type_choose, expected_output, actual_output))
    else:
        print('[FAILED]: price {},range {},type_choose {}, expect: {}, actual: {}'.format(price, down_range_spread,type_choose, expected_output, actual_output))

test_data = [
    [2.01, 5, -1, 1.95],
    [5.04, 5, -1, 4.90],
    [10.09, 5, -1, 9.75],
    [25.24, 5, -1, 24.50],
    [100.49, 5, -1, 98.75],
    [200.99, 5 , -1,197.50],
    [401.00, 5 , -1,395.00],
    [4.00, 5 , -1,3.90],
    [8.95, 5 , -1,8.70],
    [16.80, 5 , -1,16.30],
    [55.75, 5 , -1,54.50],
    [150.50, 5 , -1,148.00],
    [302.00, 5 , -1,297.00],
    [2.01, 10, -1, 1.90],
    [5.04, 10, -1, 4.80],
    [10.09, 10, -1, 9.50],
    [25.24, 10, -1, 24.00],
    [100.49, 10, -1, 97.50],
    [200.99, 10, -1,195.00],
    [401.00, 10 , -1,390.00],   
    [10.90, 5 , -1,10.40], 
    [25.25, 2 , -1,24.90], 
    [1.99, 2 , 1, 2.02],
    [9.95, 2 , 1, 10.10],
    [24.90, 2 , 1, 25.25],
    [99.75, 2 , 1, 100.50],
    [199.50, 2 , 1, 201.00],
    [399.00, 2 , 1, 402.00],
    [1.9999, 5 , 1, 2.08],
    [9.9555, 5 , 1, 10.40],
    [24.9001, 5 , 1, 26.00],
    [99.7555, 5 , 1, 102.00],
    [199.5001, 5 , 1, 204.00],
    [399.0001, 5 , 1, 408.00],
    [5.15, 20 , -1, 4.66],
    [5, 5 , 1, 5.25],
   
]

for i in range(0,len(test_data),1):  
    test_calculate_buyback_point(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3] )

[PASS]: price 2.01,range 5,type_choose -1, expect: 1.95, actual: 1.95
[PASS]: price 5.04,range 5,type_choose -1, expect: 4.9, actual: 4.9
[PASS]: price 10.09,range 5,type_choose -1, expect: 9.75, actual: 9.75
[PASS]: price 25.24,range 5,type_choose -1, expect: 24.5, actual: 24.5
[PASS]: price 100.49,range 5,type_choose -1, expect: 98.75, actual: 98.75
[PASS]: price 200.99,range 5,type_choose -1, expect: 197.5, actual: 197.5
[PASS]: price 401.0,range 5,type_choose -1, expect: 395.0, actual: 395.0
[PASS]: price 4.0,range 5,type_choose -1, expect: 3.9, actual: 3.9
[PASS]: price 8.95,range 5,type_choose -1, expect: 8.7, actual: 8.7
[PASS]: price 16.8,range 5,type_choose -1, expect: 16.3, actual: 16.3
[PASS]: price 55.75,range 5,type_choose -1, expect: 54.5, actual: 54.5
[PASS]: price 150.5,range 5,type_choose -1, expect: 148.0, actual: 148.0
[PASS]: price 302.0,range 5,type_choose -1, expect: 297.0, actual: 297.0
[PASS]: price 2.01,range 10,type_choose -1, expect: 1.9, actual: 1.9
[PASS]: 

## calculate_range_price

In [2387]:
def calculate_range_price(price,signal,count):
    # หาระยะราคาสูงสุดไปถึงราคาปิด
    list_range_price = []
    while signal >= price:
        count += 1
        list_range_price.append(round_spread(signal))
        signal = calculate_buyback_point(signal,2,-1)
        if signal < price:
            signal = calculate_buyback_point(signal,2,1)
            break
    signal = round(signal,2)       
    return list_range_price,signal,count

def test_calculate_range_price(price,signal,count, expected_price, expected_signal, expected_count):
    actual_price,actual_signal,actual_count = calculate_range_price(price,signal,count)
    if actual_price == expected_price and actual_signal == expected_signal and actual_count ==  expected_count:
        print('[PASS]: price {},signal {}, count: {},expected_price: {},actual_price: {} \n        expected_signal: {},actual_signal: {},expected_count: {},actual_count: {}'.format(price, signal, count, expected_price, actual_price, expected_signal, actual_signal, expected_count, actual_count)       
              )
    else:
        print('[FAILED]: price {},signal {}, count: {},expected_price: {},actual_price: {} \n        expected_signal: {},actual_signal: {},expected_count: {},actual_count: {}'.format(price, signal, count, expected_price, actual_price, expected_signal, actual_signal, expected_count, actual_count)       
              )
        
test_data = [
    [10, 11.8, 0,[11.8, 11.6, 11.4, 11.2, 11, 10.8, 10.6, 10.4 ,10.2, 10], 10, 10],
    [24.2, 25.25, 0,[25.25, 24.9, 24.7 ,24.5 , 24.3], 24.3, 5],
]

for i in range(0,len(test_data),1):  
    test_calculate_range_price(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3], test_data[i][4], test_data[i][5])

[PASS]: price 10,signal 11.8, count: 0,expected_price: [11.8, 11.6, 11.4, 11.2, 11, 10.8, 10.6, 10.4, 10.2, 10],actual_price: [11.8, 11.6, 11.4, 11.2, 11.0, 10.8, 10.6, 10.4, 10.2, 10.0] 
        expected_signal: 10,actual_signal: 10.0,expected_count: 10,actual_count: 10
[PASS]: price 24.2,signal 25.25, count: 0,expected_price: [25.25, 24.9, 24.7, 24.5, 24.3],actual_price: [25.25, 24.9, 24.7, 24.5, 24.3] 
        expected_signal: 24.3,actual_signal: 24.3,expected_count: 5,actual_count: 5


## calculate_cashflow

In [2388]:
def calculate_cashflow(price, volume, commission, vat, point): 
    # หากระแสเงินสดแฝงสุทธิของกองกลางและกองหลัง
    find_volume = calculate_floor_stock(volume)
    find_sell_cash = (price * find_volume) - (price * find_volume * commission) - (price * find_volume * commission * vat)
    point_buy_price = calculate_buyback_point(price,point,-1)
    find_buy_cash = (point_buy_price * find_volume) + (point_buy_price * find_volume * commission) + (point_buy_price * find_volume * commission * vat)
    cashflow = round((find_sell_cash - find_buy_cash), 2)
    return cashflow, find_volume, find_sell_cash, find_buy_cash, point_buy_price

def test_calculate_cashflow(price, volume, commission, vat, point, expected_output):
    actual_output, actual_find_volume, actual_sell_cash, actual_buy_cash, actual_point_buy_price  = calculate_cashflow(price, volume, commission, vat, point)
    if actual_output == expected_output:
        print('[PASS]: price {},volume {}, commission: {}, vat: {}, point: {},expected_output: {}, actual: {}'.format(price, volume, commission, vat, point, expected_output, actual_output))
    else:
        print('[FAILED]: price {},volume {}, commission: {}, vat: {}, point: {},expected_output: {}, actual: {}'.format(price, volume, commission, vat, point, expected_output, actual_output))

test_data = [
    [2.01, 1000, 0.00107, 0.07, 5, 55.47],
    [5.04, 1000, 0.00107, 0.07, 5, 128.62],
    [10.90, 1000, 0.00107, 0.07, 5, 475.61],
    [25.24, 1000, 0.00107, 0.07, 5, 683.05],
    [100.49, 1000, 0.00107, 0.07, 5, 1511.89],
    [200.99, 1000, 0.00107, 0.07, 5, 3033.77],
    [401.00, 1000, 0.00107, 0.07, 5, 5088.66],
    [2.01, 240, 0.00107, 0.07, 5, 11.09],
    [5.04, 240, 0.00107, 0.07, 5, 25.72],
    [10.90, 240, 0.00107, 0.07, 5, 95.12],
    [25.24, 240, 0.00107, 0.07, 5, 136.61],
    [100.49, 240, 0.00107, 0.07, 5, 302.38],
    [200.99, 240, 0.00107, 0.07, 5, 606.75],
    [401.00, 240, 0.00107, 0.07, 5, 1017.73],
]

for i in range(0,len(test_data),1):  
    test_calculate_cashflow(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3], test_data[i][4], test_data[i][5] )

[PASS]: price 2.01,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 55.47, actual: 55.47
[PASS]: price 5.04,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 128.62, actual: 128.62
[PASS]: price 10.9,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 475.61, actual: 475.61
[PASS]: price 25.24,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 683.05, actual: 683.05
[PASS]: price 100.49,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 1511.89, actual: 1511.89
[PASS]: price 200.99,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 3033.77, actual: 3033.77
[PASS]: price 401.0,volume 1000, commission: 0.00107, vat: 0.07, point: 5,expected_output: 5088.66, actual: 5088.66
[PASS]: price 2.01,volume 240, commission: 0.00107, vat: 0.07, point: 5,expected_output: 11.09, actual: 11.09
[PASS]: price 5.04,volume 240, commission: 0.00107, vat: 0.07, point: 5,expected_output: 

## calculate_net_volume

In [2389]:
def calculate_net_volume(back, mid, front, short, sum_volume):
    net_volume = 0

    # เพื่อหาจำนวนหุ้นที่ใช้ไป
    list_remaining_volume = [back, mid, front, short]
    for i in list_remaining_volume:
        for ii in i:
            net_volume += ii

    # ตรวจสอบว่ามีการซื้อหุ้นหรือยังและมีจำนวนหุ้นคงเหลือไหม
    if net_volume > 0 and sum_volume > 0:
        net_volume = sum_volume - net_volume
        net_volume_percentage = round((net_volume / sum_volume),4)
    else:
        net_volume_percentage = 1

    return net_volume_percentage , net_volume


def test_calculate_net_volume(back, mid, front, short, sum_volume, expected_volume):
    actual_remaining_net_volume , actual_remaining_net_volume_percentage = calculate_net_volume(back, mid, front, short, sum_volume)
    if actual_remaining_net_volume == expected_volume :
        print('[PASS]: list_sell_backvolume {}, list_sell_midvolume {},list_sell_frontvolume {},list_sell_savevolume {}, sum_volume: {},expected_volume: {},actual_remaining_net_volume: {} '.format(back, mid, front, short, sum_volume, expected_volume, actual_remaining_net_volume)       
              )
    else:
        print('[FAILED]: list_sell_backvolume {}, list_sell_midvolume {},list_sell_frontvolume {},list_sell_savevolume {}, sum_volume: {},expected_volume: {},actual_remaining_net_volume: {} '.format(back, mid, front, short, sum_volume, expected_volume, actual_remaining_net_volume)       
              )
        
test_data = [
    [[10667,10667,10667], [2560,2560],  [], [], 106670,0.652],
    [[], [1765,1765], [2206,4412,11029], [], 73529,0.7120],
    [[], [1765,1765], [2206,4412], [], 73529,0.8620],
    [[], [1765,1765], [2206], [], 73529,0.9220],
    [[], [1765,1765], [], [], 73529,.9520],  
    [[], [], [], [], 73529,1]
]
for i in range(0,len(test_data),1):  
    test_calculate_net_volume(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3], test_data[i][4], test_data[i][5])

[PASS]: list_sell_backvolume [10667, 10667, 10667], list_sell_midvolume [2560, 2560],list_sell_frontvolume [],list_sell_savevolume [], sum_volume: 106670,expected_volume: 0.652,actual_remaining_net_volume: 0.652 
[PASS]: list_sell_backvolume [], list_sell_midvolume [1765, 1765],list_sell_frontvolume [2206, 4412, 11029],list_sell_savevolume [], sum_volume: 73529,expected_volume: 0.712,actual_remaining_net_volume: 0.712 
[PASS]: list_sell_backvolume [], list_sell_midvolume [1765, 1765],list_sell_frontvolume [2206, 4412],list_sell_savevolume [], sum_volume: 73529,expected_volume: 0.862,actual_remaining_net_volume: 0.862 
[PASS]: list_sell_backvolume [], list_sell_midvolume [1765, 1765],list_sell_frontvolume [2206],list_sell_savevolume [], sum_volume: 73529,expected_volume: 0.922,actual_remaining_net_volume: 0.922 
[PASS]: list_sell_backvolume [], list_sell_midvolume [1765, 1765],list_sell_frontvolume [],list_sell_savevolume [], sum_volume: 73529,expected_volume: 0.952,actual_remaining_net

## update_trading_history_back

In [2390]:
def update_trading_history_back(trade_history_df, stock_vars, index):

    trade_history_df.loc[index, 'price_buy'] = trade_history_df.loc[index, 'point_buyback']
    trade_history_df.loc[index, 'volume_buy'] = trade_history_df.loc[index, 'volume_sell']
    trade_history_df.loc[index, 'cash_buy'] = trade_history_df.loc[index, 'cash']
    trade_history_df.loc[index, 'cash'] = 0.00
    stock_vars['list_data_index_back'].pop(0)

## update_trading_history_front

In [2391]:
def update_trading_history_front(trade_history_df, stock_vars, index):

    trade_history_df.loc[index, 'price_buy'] = trade_history_df.loc[index, 'point_buyback']
    trade_history_df.loc[index, 'volume_buy'] = trade_history_df.loc[index, 'volume_sell']
    trade_history_df.loc[index, 'cash_buy'] = trade_history_df.loc[index, 'cash']
    trade_history_df.loc[index, 'cash'] = 0.00


## update_cashflows

In [2392]:
def update_cashflows(stock_vars, keep_cashflow, percentages, type_strategy):
    # อัพเดตกระแสเงินสดที่ได้มา

    if type_strategy == 'กองหลัง':
        stock_vars['cashflow_back'] += keep_cashflow
        dict_all_sum_port['sum_cashflow_back'] += keep_cashflow
    elif type_strategy == 'กองกลาง':
        stock_vars['cashflow_mid'] += keep_cashflow
        dict_all_sum_port['sum_cashflow_mid'] += keep_cashflow
    elif type_strategy == 'กองหน้า':
        stock_vars['cashflow_front'] += keep_cashflow
        dict_all_sum_port['sum_cashflow_front'] += keep_cashflow
    elif type_strategy == 'แปลงร่าง':
        stock_vars['cashflow_convert'] += keep_cashflow
        dict_all_sum_port['sum_cashflow_convert'] += keep_cashflow
    elif type_strategy == 'short':
        stock_vars['cashflow_short'] += keep_cashflow
        dict_all_sum_port['sum_cashflow_short'] += keep_cashflow



    # PORT
    dict_all_sum_port['cashflow_buymore'] += keep_cashflow * percentages['buymore']
    dict_all_sum_port['cashflow_reserve'] += keep_cashflow * percentages['reserve']
    dict_all_sum_port['cashflow_fee'] += keep_cashflow * percentages['fee']


## update_trade_hist

In [2393]:
def update_trade_hist(date, time, name_type, sum_volume, increase_volume, net_volume, avg_price, price_sell, volume_sell, price_buy, volume_buy, point_buyback, cash_sell, cash_buy, cash, cashflow_back, cashflow_mid, cashflow_front, cashflow_convert, cashflow_short, cashflow_buymore):
    new_data = {
        'date': date,
        'time': time,
        'type': name_type,
        "avg_price": avg_price ,
        "sum_volume": sum_volume,
        "increase_volume": increase_volume,
        "net_volume": net_volume,
        'price_sell': price_sell,
        'volume_sell': volume_sell,
        'price_buy': price_buy,
        'volume_buy': volume_buy,                                
        'point_buyback': point_buyback,
        'cash_sell': cash_sell,
        'cash_buy': cash_buy,
        'cash': cash,
        'cashflow_back': cashflow_back,
        'cashflow_mid': cashflow_mid,
        'cashflow_front': cashflow_front, 
        'cashflow_convert': cashflow_convert,
        'cashflow_short': cashflow_short,
        'cashflow_buymore': cashflow_buymore,                            
    } 
    return pd.DataFrame([new_data])



## draw_setting_fibo

In [2394]:
def draw_setting_pricefibo(list_level_pricefibo):
    fibo_level = [1.618, 2.618 , 3.618, 4.236, 5.545]
    for level in fibo_level:
        price = (list_level_pricefibo[0] + ((list_level_pricefibo[1] - list_level_pricefibo[0]) * level))
        spread = calculate_spread(price)
        lower_bound = price - (price % spread)
        upper_bound = lower_bound + spread
        if (price - lower_bound) < (upper_bound - price):
            list_level_pricefibo.append(round(lower_bound, 2))
        else:
            list_level_pricefibo.append(round(upper_bound, 2))
    return list_level_pricefibo

def test_draw_setting_fibo(list_level_pricefibo, expected_output):
    actual_output = draw_setting_pricefibo(list_level_pricefibo)
    if actual_output == expected_output:
        print('[PASS]: list_level_pricefibo {}, expect: {}, actual: {}'.format(list_level_pricefibo, expected_output, actual_output))
    else:
        print('[FAILED]: list_level_pricefibo {}, expect: {}, actual: {}'.format(list_level_pricefibo, expected_output, actual_output))     

test_data = [
    [[4.66, 5.15], [4.66, 5.15, 5.45, 5.95, 6.45, 6.75, 7.4]],
    [[60.5, 65.5], [60.5, 65.5, 68.5, 73.5, 78.5, 81.75, 88.25]],
    [[222.0, 242.0], [222.0, 242.0, 254.0, 274.0, 294.0, 307.0, 333.0]],
]

for i in range(0,len(test_data),1):  
    test_draw_setting_fibo(test_data[i][0],test_data[i][1] )


[PASS]: list_level_pricefibo [4.66, 5.15, 5.45, 5.95, 6.45, 6.75, 7.4], expect: [4.66, 5.15, 5.45, 5.95, 6.45, 6.75, 7.4], actual: [4.66, 5.15, 5.45, 5.95, 6.45, 6.75, 7.4]
[PASS]: list_level_pricefibo [60.5, 65.5, 68.5, 73.5, 78.5, 81.75, 88.25], expect: [60.5, 65.5, 68.5, 73.5, 78.5, 81.75, 88.25], actual: [60.5, 65.5, 68.5, 73.5, 78.5, 81.75, 88.25]
[PASS]: list_level_pricefibo [222.0, 242.0, 254.0, 274.0, 294.0, 307.0, 333.0], expect: [222.0, 242.0, 254.0, 274.0, 294.0, 307.0, 333.0], actual: [222.0, 242.0, 254.0, 274.0, 294.0, 307.0, 333.0]


## calculate_midband_cashflow

In [2395]:
def calculate_midband_cashflow(list_sell_midprice, list_sell_midvolume, list_avgcost_pricefibo, COMMISSION, VAT):
    # หากระแสเงินสดแฝงกองกลางที่จุดซื้อคืน fibo level 100
    calculate_sell_cashflow = (list_sell_midprice[0] * list_sell_midvolume[0]) - (list_sell_midprice[0] * list_sell_midvolume[0] * COMMISSION) - (list_sell_midprice[0] * list_sell_midvolume[0] * COMMISSION * VAT)
    calculate_buy_cashflow = (list_avgcost_pricefibo[-1] * list_sell_midvolume[0]) + (list_avgcost_pricefibo[-1] * list_sell_midvolume[0] * COMMISSION) + (list_avgcost_pricefibo[-1] * list_sell_midvolume[0] * COMMISSION * VAT)
    net_cashflow = round((calculate_sell_cashflow - calculate_buy_cashflow), 2)
    return net_cashflow, calculate_sell_cashflow, calculate_buy_cashflow

def test_calculate_midband_cashflow(list_sell_midprice, list_sell_midvolume, list_avgcost_pricefibo, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow, actual_sell_cashflow, actual_buy_cashflow = calculate_midband_cashflow(list_sell_midprice, list_sell_midvolume, list_avgcost_pricefibo, COMMISSION, VAT)
    if actual_net_cashflow == expected_net_cashflow:
        print('[PASS]: list_sell_midprice {}, list_sell_midvolume {},list_avgcost_pricefibo: {},COMMISSION: {},VAT: {} \n        expected_cashflow: {},actual_cashflow: {}'.format(list_sell_midprice, list_sell_midvolume, list_avgcost_pricefibo, COMMISSION, VAT, expected_net_cashflow, actual_net_cashflow)       
              )
    else:
        print('[FAILED]: list_sell_midprice {}, list_sell_midvolume {},list_avgcost_pricefibo: {},COMMISSION: {},VAT: {} \n        expected_cashflow: {},actual_cashflow: {}'.format(list_sell_midprice, list_sell_midvolume, list_avgcost_pricefibo, COMMISSION, VAT, expected_net_cashflow, actual_net_cashflow)       
              )

test_data = [
    [[61], [300], [58], 0.00107, 0.07, 859.13],
    [[64.75], [300], [60.5], 0.00107, 0.07, 1231.98],
    [[53.50], [300], [44], 0.00107, 0.07, 2816.51],
    [[83.75], [300], [78.5], 0.00107, 0.07, 1519.27],
]

for i in range(0,len(test_data),1):  
    test_calculate_midband_cashflow(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3], test_data[i][4], test_data[i][5])

[PASS]: list_sell_midprice [61], list_sell_midvolume [300],list_avgcost_pricefibo: [58],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 859.13,actual_cashflow: 859.13
[PASS]: list_sell_midprice [64.75], list_sell_midvolume [300],list_avgcost_pricefibo: [60.5],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 1231.98,actual_cashflow: 1231.98
[PASS]: list_sell_midprice [53.5], list_sell_midvolume [300],list_avgcost_pricefibo: [44],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 2816.51,actual_cashflow: 2816.51
[PASS]: list_sell_midprice [83.75], list_sell_midvolume [300],list_avgcost_pricefibo: [78.5],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 1519.27,actual_cashflow: 1519.27


## calculate_short_cashflow

In [2396]:
def calculate_short_cashflow(list_sell_shortprice, list_sell_shortvolume, list_buy_shortprice, COMMISSION, VAT):
    # หากระแสเงินสดระบบ short
    calculate_sell_cashflow = (list_sell_shortprice[0] * list_sell_shortvolume[0]) - (list_sell_shortprice[0] * list_sell_shortvolume[0] * COMMISSION) - (list_sell_shortprice[0] * list_sell_shortvolume[0] * COMMISSION * VAT)
    calculate_buy_cashflow = (list_buy_shortprice[0] * list_sell_shortvolume[0]) + (list_buy_shortprice[0] * list_sell_shortvolume[0] * COMMISSION) + (list_buy_shortprice[0] * list_sell_shortvolume[0] * COMMISSION * VAT)
    net_cashflow = round((calculate_sell_cashflow - calculate_buy_cashflow), 2)
    return net_cashflow, calculate_sell_cashflow, calculate_buy_cashflow

def test_calculate_short_cashflow(list_sell_shortprice, list_sell_shortvolume, list_buy_shortprice, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow, actual_sell_cashflow, actual_buy_cashflow = calculate_short_cashflow(list_sell_shortprice, list_sell_shortvolume, list_buy_shortprice, COMMISSION, VAT)
    if actual_net_cashflow == expected_net_cashflow:
        print('[PASS]: list_sell_shortprice {}, list_sell_shortvolume {},list_buy_shortprice: {},COMMISSION: {},VAT: {} \n        expected_cashflow: {},actual_cashflow: {}'.format(list_sell_shortprice, list_sell_shortvolume, list_buy_shortprice, COMMISSION, VAT, expected_net_cashflow, actual_net_cashflow)       
              )
    else:
        print('[FAILED]: list_sell_shortprice {}, list_sell_shortvolume {},list_buy_shortprice: {},COMMISSION: {},VAT: {} \n        expected_cashflow: {},actual_cashflow: {}'.format(list_sell_shortprice, list_sell_shortvolume, list_buy_shortprice, COMMISSION, VAT, expected_net_cashflow, actual_net_cashflow)       
              )

test_data = [
    [[61], [300], [58], 0.00107, 0.07, 859.13],
    [[64.75], [300], [60.5], 0.00107, 0.07, 1231.98],
    [[53.50], [300], [44], 0.00107, 0.07, 2816.51],
    [[83.75], [300], [78.5], 0.00107, 0.07, 1519.27],
]

for i in range(0,len(test_data),1):  
    test_calculate_short_cashflow(test_data[i][0], test_data[i][1], test_data[i][2], test_data[i][3], test_data[i][4], test_data[i][5])


[PASS]: list_sell_shortprice [61], list_sell_shortvolume [300],list_buy_shortprice: [58],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 859.13,actual_cashflow: 859.13
[PASS]: list_sell_shortprice [64.75], list_sell_shortvolume [300],list_buy_shortprice: [60.5],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 1231.98,actual_cashflow: 1231.98
[PASS]: list_sell_shortprice [53.5], list_sell_shortvolume [300],list_buy_shortprice: [44],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 2816.51,actual_cashflow: 2816.51
[PASS]: list_sell_shortprice [83.75], list_sell_shortvolume [300],list_buy_shortprice: [78.5],COMMISSION: 0.00107,VAT: 0.07 
        expected_cashflow: 1519.27,actual_cashflow: 1519.27


## calculate_min_sellvolume

In [2397]:
def calculate_min_sellvolume(net_sell_midvolume, list_sell_volume, PERCENTAGE, SUM_PERCENTAGE):
    # คำนวณปริมาณขายขั้นต่ำที่ต้องการตามปริมาณรวมและเปอร์เซ็นต์ที่กำหนด
    keep_min_sellvolume = max(calculate_floor_stock(net_sell_midvolume * PERCENTAGE), 100)
    
    # คำนวณผลรวมของปริมาณขายที่มีใน list_sell_midvolume
    total_sell_volume = sum(list_sell_volume)

    # ตัดสินใจหาปริมาณขายขั้นต่ำที่เป็นไปได้
    # โดยเปรียบเทียบระหว่างปริมาณขายขั้นต่ำที่ต้องการกับปริมาณขายที่เป็นไปได้ภายในข้อจำกัดที่กำหนด
    potential_sell_volume = net_sell_midvolume - total_sell_volume
    find_min_sellvolume = calculate_floor_stock(min(keep_min_sellvolume, potential_sell_volume))

    # ตรวจสอบว่าเกินจำนวนที่จำกัดหรือไม่
    if (calculate_floor_stock(net_sell_midvolume * SUM_PERCENTAGE) - (total_sell_volume + find_min_sellvolume)) >= 0:
        return find_min_sellvolume
    else:
 
        if (calculate_floor_stock(net_sell_midvolume * SUM_PERCENTAGE) - total_sell_volume ) >= 100:
            find_min_sellvolume = calculate_floor_stock((net_sell_midvolume * SUM_PERCENTAGE) - total_sell_volume )
            return find_min_sellvolume
        else:
            find_min_sellvolume = 0
            return find_min_sellvolume



def test_calculate_min_sellvolume(net_sell_midvolume, list_sell_volume, PERCENTAGE, SUM_PERCENTAGE, expected_result):
    actual_result = calculate_min_sellvolume(net_sell_midvolume, list_sell_volume, PERCENTAGE, SUM_PERCENTAGE)
    if actual_result == expected_result:
        print(f'[PASS]: sum_volume {net_sell_midvolume}, list_sell_volume {list_sell_volume}, '
              f'PERCENTAGE {PERCENTAGE}, SUM_PERCENTAGE {SUM_PERCENTAGE},  \n'
              f'       expected_result: {expected_result}, actual_result: {actual_result}')
    else:
        print(f'[FAILED]: sum_volume {net_sell_midvolume}, list_sell_volume {list_sell_volume}, '
              f'PERCENTAGE {PERCENTAGE}, SUM_PERCENTAGE {SUM_PERCENTAGE},  \n'
              f'       expected_result: {expected_result}, actual_result: {actual_result}')

test_data = [
    [4500, [300, 600, 900, 1200], 0.05, .79, 200], 
    [3000, [200, 400, 600], 0.14, .79, 300],        
    [6000, [1000, 2000, 3000], 0.05, .79, 0],    
    [10000, [400, 800, 1200, 1600], 0.32, .79, 3200], 
    [5000, [200, 300, 400], 0.10, .21, 100]        
]


for test_case in test_data:
    test_calculate_min_sellvolume(*test_case)

[PASS]: sum_volume 4500, list_sell_volume [300, 600, 900, 1200], PERCENTAGE 0.05, SUM_PERCENTAGE 0.79,  
       expected_result: 200, actual_result: 200
[FAILED]: sum_volume 3000, list_sell_volume [200, 400, 600], PERCENTAGE 0.14, SUM_PERCENTAGE 0.79,  
       expected_result: 300, actual_result: 400
[PASS]: sum_volume 6000, list_sell_volume [1000, 2000, 3000], PERCENTAGE 0.05, SUM_PERCENTAGE 0.79,  
       expected_result: 0, actual_result: 0
[PASS]: sum_volume 10000, list_sell_volume [400, 800, 1200, 1600], PERCENTAGE 0.32, SUM_PERCENTAGE 0.79,  
       expected_result: 3200, actual_result: 3200
[PASS]: sum_volume 5000, list_sell_volume [200, 300, 400], PERCENTAGE 0.1, SUM_PERCENTAGE 0.21,  
       expected_result: 100, actual_result: 100


## calculate_max_volumesell

In [2398]:
def calculate_max_volumesell(net_sell_volume, PERCENTAGE, SUM_PERCENTAGE):
    list_volume = []

    # หาจำนวนหุ้นที่ขายได้สูงสุด
    max_volume = net_sell_volume * SUM_PERCENTAGE

    # คำนวณปริมาณขายขั้นต่ำที่ต้องการตามปริมาณรวมและเปอร์เซ็นต์ที่กำหนด
    for i in PERCENTAGE:
        keep_min_sellvolume = max(calculate_floor_stock(net_sell_volume * i), 100)

        if (max_volume - keep_min_sellvolume) > 0:
            list_volume.append(keep_min_sellvolume)
            max_volume -= keep_min_sellvolume
        else:
            list_volume.append(calculate_floor_stock(max_volume))
            break
    
    return list_volume        

def test_calculate_max_volumesell(net_sell_volume, PERCENTAGE, SUM_PERCENTAGE, expected_volumes):
    actual_volumes = calculate_max_volumesell(net_sell_volume, PERCENTAGE, SUM_PERCENTAGE)
    if actual_volumes == expected_volumes:
        print(f'[PASS]: net_sell_volume {net_sell_volume}, PERCENTAGE {PERCENTAGE}, SUM_PERCENTAGE {SUM_PERCENTAGE}\n' +
              f'        expected_volumes: {expected_volumes}, actual_volumes: {actual_volumes}')
    else:
        print(f'[FAILED]: net_sell_volume {net_sell_volume}, PERCENTAGE {PERCENTAGE}, SUM_PERCENTAGE {SUM_PERCENTAGE}\n' +
              f'        expected_volumes: {expected_volumes}, actual_volumes: {actual_volumes}')


test_data = [
    [3000,[0.0125, 0.0251, 0.0502, 0.1003, 0.2006, 0.4013] , .79, [100, 100, 100, 300, 600, 1100]],        
    [6000,[0.0125, 0.0251, 0.0502, 0.1003, 0.2006, 0.4013] , .79, [100, 100, 300, 600, 1200, 2400]],    
    [10000,[0.01, 0.02, 0.03, 0.04, 0.05, 0.06], .20, [100, 200, 300, 400, 500, 500]], 
    [5000,[0.01, 0.02, 0.03, 0.04, 0.05, 0.06],  .12, [100, 100, 100, 200, 100]]       
]


for test_case in test_data:
    test_calculate_max_volumesell(*test_case)

[PASS]: net_sell_volume 3000, PERCENTAGE [0.0125, 0.0251, 0.0502, 0.1003, 0.2006, 0.4013], SUM_PERCENTAGE 0.79
        expected_volumes: [100, 100, 100, 300, 600, 1100], actual_volumes: [100, 100, 100, 300, 600, 1100]
[PASS]: net_sell_volume 6000, PERCENTAGE [0.0125, 0.0251, 0.0502, 0.1003, 0.2006, 0.4013], SUM_PERCENTAGE 0.79
        expected_volumes: [100, 100, 300, 600, 1200, 2400], actual_volumes: [100, 100, 300, 600, 1200, 2400]
[PASS]: net_sell_volume 10000, PERCENTAGE [0.01, 0.02, 0.03, 0.04, 0.05, 0.06], SUM_PERCENTAGE 0.2
        expected_volumes: [100, 200, 300, 400, 500, 500], actual_volumes: [100, 200, 300, 400, 500, 500]
[PASS]: net_sell_volume 5000, PERCENTAGE [0.01, 0.02, 0.03, 0.04, 0.05, 0.06], SUM_PERCENTAGE 0.12
        expected_volumes: [100, 100, 100, 200, 100], actual_volumes: [100, 100, 100, 200, 100]


## calculate_avgcost_pricefibo

In [2399]:
def calculate_avgcost_pricefibo(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo):

    # หาจำนวนหุ้นที่ขายใน fibo level 161.8
    volume_level_fibo2 = list_level_volumefibo[0] + list_level_volumefibo[1]
    price_level_fibo2 = (list_level_pricefibo[1] * list_level_volumefibo[0]) + (list_level_pricefibo[2] * list_level_volumefibo[1])
    list_avgcost_pricefibo.append(calculate_buyback_point((price_level_fibo2/volume_level_fibo2), 5, -1))

    # หาจำนวนหุ้นที่ขายใน fibo level 261.8
    volume_level_fibo3 = list_level_volumefibo[0] + list_level_volumefibo[1] + list_level_volumefibo[2]
    price_level_fibo3 = (list_level_pricefibo[1] * list_level_volumefibo[0]) + (list_level_pricefibo[2] * list_level_volumefibo[1]) + (list_level_pricefibo[3] * list_level_volumefibo[2])
    list_avgcost_pricefibo.append(calculate_buyback_point((price_level_fibo3/volume_level_fibo3), 5, -1))   

    # หาจำนวนหุ้นที่ขายใน fibo level 361.8
    volume_level_fibo4 = list_level_volumefibo[0] + list_level_volumefibo[1] + list_level_volumefibo[2] + list_level_volumefibo[3]
    price_level_fibo4 = (list_level_pricefibo[1] * list_level_volumefibo[0]) + (list_level_pricefibo[2] * list_level_volumefibo[1]) + (list_level_pricefibo[3] * list_level_volumefibo[2]) + (list_level_pricefibo[4] * list_level_volumefibo[3])
    list_avgcost_pricefibo.append(calculate_buyback_point((price_level_fibo4/volume_level_fibo4), 5, -1))   

    # หาจำนวนหุ้นที่ขายใน fibo level 423.6
    volume_level_fibo5 = list_level_volumefibo[0] + list_level_volumefibo[1] + list_level_volumefibo[2] + list_level_volumefibo[3] + list_level_volumefibo[4]
    price_level_fibo5 = (list_level_pricefibo[1] * list_level_volumefibo[0]) + (list_level_pricefibo[2] * list_level_volumefibo[1]) + (list_level_pricefibo[3] * list_level_volumefibo[2]) + (list_level_pricefibo[4] * list_level_volumefibo[3]) + (list_level_pricefibo[5] * list_level_volumefibo[4])
    list_avgcost_pricefibo.append(calculate_buyback_point((price_level_fibo5/volume_level_fibo5), 5, -1))   

    # หาจำนวนหุ้นที่ขายใน fibo level 545.5
    volume_level_fibo6 = list_level_volumefibo[0] + list_level_volumefibo[1] + list_level_volumefibo[2] + list_level_volumefibo[3] + list_level_volumefibo[4] + list_level_volumefibo[5]
    price_level_fibo6 = (list_level_pricefibo[1] * list_level_volumefibo[0]) + (list_level_pricefibo[2] * list_level_volumefibo[1]) + (list_level_pricefibo[3] * list_level_volumefibo[2]) + (list_level_pricefibo[4] * list_level_volumefibo[3]) + (list_level_pricefibo[5] * list_level_volumefibo[4]) + (list_level_pricefibo[6] * list_level_volumefibo[5])
    list_avgcost_pricefibo.append(calculate_buyback_point((price_level_fibo6/volume_level_fibo6), 5, -1))      

def test_calculate_avgcost_pricefibo(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, expected_results):
    calculate_avgcost_pricefibo(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo)
    if list_avgcost_pricefibo == expected_results:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo} \n       expected_results: {expected_results}, actual_results: {list_avgcost_pricefibo}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo} \n       expected_results: {expected_results}, actual_results: {list_avgcost_pricefibo}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[22.9, 24.9, 26.25, 28.25, 30.25, 31.25, 34.0], [100, 300, 900, 2300, 4800, 10700], [], [24.8, 26.25, 28.0, 29.0, 31.0]], 
    [[4.5, 4.9, 5.15, 5.55, 5.95, 6.2, 6.7], [900, 1900, 4900, 11900, 24900, 54900], [], [4.92, 5.1, 5.45, 5.7, 6.1]], 
    [[162.0, 172.0, 178.0, 188.0, 198.0, 204.0, 217.0], [100, 100, 100, 300, 700, 1600], [], [172.5, 176.5, 186.0, 194.0, 203.0]], 
]

# รัน unit tests
for test_case in test_data:
    test_calculate_avgcost_pricefibo(*test_case)
  

[PASS]: list_level_pricefibo [22.9, 24.9, 26.25, 28.25, 30.25, 31.25, 34.0], list_level_volumefibo [100, 300, 900, 2300, 4800, 10700] 
       expected_results: [24.8, 26.25, 28.0, 29.0, 31.0], actual_results: [24.8, 26.25, 28.0, 29.0, 31.0]
[PASS]: list_level_pricefibo [4.5, 4.9, 5.15, 5.55, 5.95, 6.2, 6.7], list_level_volumefibo [900, 1900, 4900, 11900, 24900, 54900] 
       expected_results: [4.92, 5.1, 5.45, 5.7, 6.1], actual_results: [4.92, 5.1, 5.45, 5.7, 6.1]
[PASS]: list_level_pricefibo [162.0, 172.0, 178.0, 188.0, 198.0, 204.0, 217.0], list_level_volumefibo [100, 100, 100, 300, 700, 1600] 
       expected_results: [172.5, 176.5, 186.0, 194.0, 203.0], actual_results: [172.5, 176.5, 186.0, 194.0, 203.0]


## update_clear_datamid

In [2400]:
def update_clear_datamid(list_data_index_mid, list_sell_midprice, list_sell_midvolume, list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_dif_midcashflow, list_level_volumefront):
        list_data_index_mid.clear() 
        list_sell_midprice.clear() 
        list_sell_midvolume.clear() 
        list_level_pricefibo.clear() 
        list_level_volumefibo.clear() 
        list_avgcost_pricefibo.clear() 
        list_buy_midcashflow.clear() 
        list_dif_midcashflow.clear()
        list_level_volumefront.clear()
        

## calculate_tp_pricefibo2_cashflow

In [2401]:
def calculate_tp_pricefibo2_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT):

    # หาเงินสดฝั่งขายรวม fibo level 161.8
    list_sell_midcashflow.clear()
    list_sell_midcashflow.append((list_level_pricefibo[1] * list_level_volumefibo[0]) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[2] * list_level_volumefibo[1]) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))

    # หาเงินสดฝั่งซื้อคืนรวม fibo level 161.8
    list_buy_midcashflow.clear()
    list_buy_midcashflow.append((list_avgcost_pricefibo[0] * list_level_volumefibo[0]) + (list_avgcost_pricefibo[0] * list_level_volumefibo[0] * COMMISSION) + (list_avgcost_pricefibo[0] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[0] * list_level_volumefibo[1]) + (list_avgcost_pricefibo[0] * list_level_volumefibo[1] * COMMISSION) + (list_avgcost_pricefibo[0] * list_level_volumefibo[1] * COMMISSION * VAT))

    # หากระแสเงินสดแฝงได้รับจาก fibo level 161.8
    net_cashflow = round((sum(list_sell_midcashflow)- sum(list_buy_midcashflow)), 2)
    return net_cashflow


def test_calculate_tp_pricefibo2_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow = calculate_tp_pricefibo2_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[162.0, 172.0, 178.0], [100, 100, 100], [172.5], [], [], 0.00107, 0.07, 420.43],  
    [[177.5, 187.5, 193.5], [100, 100, 100], [188], [], [], 0.00107, 0.07, 413.33],     
    [[9.7, 11.4, 12.5], [600, 1200, 3100], [11.6], [], [], 0.00107, 0.07, 911.09],   
]

# รัน unit tests
for test_case in test_data:
    test_calculate_tp_pricefibo2_cashflow(*test_case)


[PASS]: list_level_pricefibo [162.0, 172.0, 178.0], list_level_volumefibo [100, 100, 100], list_avgcost_pricefibo [172.5], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 420.43, actual_net_cashflow: 420.43
[PASS]: list_level_pricefibo [177.5, 187.5, 193.5], list_level_volumefibo [100, 100, 100], list_avgcost_pricefibo [188], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 413.33, actual_net_cashflow: 413.33
[PASS]: list_level_pricefibo [9.7, 11.4, 12.5], list_level_volumefibo [600, 1200, 3100], list_avgcost_pricefibo [11.6], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 911.09, actual_net_cashflow: 911.09


## calculate_tp_pricefibo3_cashflow

In [2402]:
def calculate_tp_pricefibo3_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT):

    # หาเงินสดฝั่งขายรวม fibo level 261.8
    list_sell_midcashflow.clear()
    list_sell_midcashflow.append((list_level_pricefibo[1] * list_level_volumefibo[0]) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[2] * list_level_volumefibo[1]) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[3] * list_level_volumefibo[2]) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION * VAT))

    # หาเงินสดฝั่งซื้อคืนรวม fibo level 261.8
    list_buy_midcashflow.clear()
    list_buy_midcashflow.append((list_avgcost_pricefibo[1] * list_level_volumefibo[0]) + (list_avgcost_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) + (list_avgcost_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[1] * list_level_volumefibo[1]) + (list_avgcost_pricefibo[1] * list_level_volumefibo[1] * COMMISSION) + (list_avgcost_pricefibo[1] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[1] * list_level_volumefibo[2]) + (list_avgcost_pricefibo[1] * list_level_volumefibo[2] * COMMISSION) + (list_avgcost_pricefibo[1] * list_level_volumefibo[2] * COMMISSION * VAT))

    # หากระแสเงินสดแฝงได้รับจาก fibo level 261.8
    net_cashflow = round((sum(list_sell_midcashflow)- sum(list_buy_midcashflow)), 2)
    return net_cashflow


def test_calculate_tp_pricefibo3_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow = calculate_tp_pricefibo3_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], [100, 100, 300, 700, 1600, 3500], [61.75, 65.5, 70.0, 73.75, 78.75], [], [], 0.00107, 0.07, 624.21],  
    [[177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], [100, 100, 100, 200, 500, 1200], [188.0, 192.5, 198.5, 206.0, 218.0], [], [], 0.00107, 0.07, 616.91],     
    [[7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], [400, 800, 2200, 5200, 11000, 24200], [8.25, 9.0, 9.8, 10.2, 11.2], [], [], 0.00107, 0.07, 868.86],   
]

# รัน unit tests
for test_case in test_data:
    test_calculate_tp_pricefibo3_cashflow(*test_case)


[PASS]: list_level_pricefibo [56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], list_level_volumefibo [100, 100, 300, 700, 1600, 3500], list_avgcost_pricefibo [61.75, 65.5, 70.0, 73.75, 78.75], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 624.21, actual_net_cashflow: 624.21
[PASS]: list_level_pricefibo [177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], list_level_volumefibo [100, 100, 100, 200, 500, 1200], list_avgcost_pricefibo [188.0, 192.5, 198.5, 206.0, 218.0], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 616.91, actual_net_cashflow: 616.91
[PASS]: list_level_pricefibo [7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], list_level_volumefibo [400, 800, 2200, 5200, 11000, 24200], list_avgcost_pricefibo [8.25, 9.0, 9.8, 10.2, 11.2], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 868.86, actual_net_cashflow: 868.86


## calculate_tp_pricefibo4_cashflow

In [2403]:
def calculate_tp_pricefibo4_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT):

    # หาเงินสดฝั่งขายรวม fibo level 361.8
    list_sell_midcashflow.clear()
    list_sell_midcashflow.append((list_level_pricefibo[1] * list_level_volumefibo[0]) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[2] * list_level_volumefibo[1]) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[3] * list_level_volumefibo[2]) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[4] * list_level_volumefibo[3]) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION * VAT))

    # หาเงินสดฝั่งซื้อคืนรวม fibo level 361.8
    list_buy_midcashflow.clear()
    list_buy_midcashflow.append((list_avgcost_pricefibo[2] * list_level_volumefibo[0]) + (list_avgcost_pricefibo[2] * list_level_volumefibo[0] * COMMISSION) + (list_avgcost_pricefibo[2] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[2] * list_level_volumefibo[1]) + (list_avgcost_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) + (list_avgcost_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[2] * list_level_volumefibo[2]) + (list_avgcost_pricefibo[2] * list_level_volumefibo[2] * COMMISSION) + (list_avgcost_pricefibo[2] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[2] * list_level_volumefibo[3]) + (list_avgcost_pricefibo[2] * list_level_volumefibo[3] * COMMISSION) + (list_avgcost_pricefibo[2] * list_level_volumefibo[3] * COMMISSION * VAT))

    # หากระแสเงินสดแฝงได้รับจาก fibo level 361.8
    net_cashflow = round((sum(list_sell_midcashflow)- sum(list_buy_midcashflow)), 2)
    return net_cashflow


def test_calculate_tp_pricefibo4_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow = calculate_tp_pricefibo4_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], [100, 100, 300, 700, 1600, 3500], [61.75, 65.5, 70.0, 73.75, 78.75], [], [], 0.00107, 0.07, 1405.82],  
    [[177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], [100, 100, 100, 200, 500, 1200], [188.0, 192.5, 198.5, 206.0, 218.0], [], [], 0.00107, 0.07, 1820.39],     
    [[7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], [400, 800, 2200, 5200, 11000, 24200], [8.25, 9.0, 9.8, 10.2, 11.2], [], [], 0.00107, 0.07, 2703.70],   
]

# รัน unit tests
for test_case in test_data:
    test_calculate_tp_pricefibo4_cashflow(*test_case)


[PASS]: list_level_pricefibo [56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], list_level_volumefibo [100, 100, 300, 700, 1600, 3500], list_avgcost_pricefibo [61.75, 65.5, 70.0, 73.75, 78.75], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 1405.82, actual_net_cashflow: 1405.82
[PASS]: list_level_pricefibo [177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], list_level_volumefibo [100, 100, 100, 200, 500, 1200], list_avgcost_pricefibo [188.0, 192.5, 198.5, 206.0, 218.0], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 1820.39, actual_net_cashflow: 1820.39
[PASS]: list_level_pricefibo [7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], list_level_volumefibo [400, 800, 2200, 5200, 11000, 24200], list_avgcost_pricefibo [8.25, 9.0, 9.8, 10.2, 11.2], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 2703.7, actual_net_cashflow: 2703.7


## calculate_tp_pricefibo5_cashflow

In [2404]:
def calculate_tp_pricefibo5_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT):

    # หาเงินสดฝั่งขายรวม fibo level 423.6
    list_sell_midcashflow.clear()
    list_sell_midcashflow.append((list_level_pricefibo[1] * list_level_volumefibo[0]) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[2] * list_level_volumefibo[1]) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[3] * list_level_volumefibo[2]) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[4] * list_level_volumefibo[3]) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[5] * list_level_volumefibo[4]) - (list_level_pricefibo[5] * list_level_volumefibo[4] * COMMISSION) - (list_level_pricefibo[5] * list_level_volumefibo[4] * COMMISSION * VAT))

    # หาเงินสดฝั่งซื้อคืนรวม fibo level 423.6
    list_buy_midcashflow.clear()
    list_buy_midcashflow.append((list_avgcost_pricefibo[3] * list_level_volumefibo[0]) + (list_avgcost_pricefibo[3] * list_level_volumefibo[0] * COMMISSION) + (list_avgcost_pricefibo[3] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[3] * list_level_volumefibo[1]) + (list_avgcost_pricefibo[3] * list_level_volumefibo[1] * COMMISSION) + (list_avgcost_pricefibo[3] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[3] * list_level_volumefibo[2]) + (list_avgcost_pricefibo[3] * list_level_volumefibo[2] * COMMISSION) + (list_avgcost_pricefibo[3] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[3] * list_level_volumefibo[3]) + (list_avgcost_pricefibo[3] * list_level_volumefibo[3] * COMMISSION) + (list_avgcost_pricefibo[3] * list_level_volumefibo[3] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[3] * list_level_volumefibo[4]) + (list_avgcost_pricefibo[3] * list_level_volumefibo[4] * COMMISSION) + (list_avgcost_pricefibo[3] * list_level_volumefibo[4] * COMMISSION * VAT))

    # หากระแสเงินสดแฝงได้รับจาก fibo level 423.6
    net_cashflow = round((sum(list_sell_midcashflow)- sum(list_buy_midcashflow)), 2)
    return net_cashflow


def test_calculate_tp_pricefibo5_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow = calculate_tp_pricefibo5_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], [100, 100, 300, 700, 1600, 3500], [61.75, 65.5, 70.0, 73.75, 78.75], [], [], 0.00107, 0.07, 3023.15],  
    [[177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], [100, 100, 100, 200, 500, 1200], [188.0, 192.5, 198.5, 206.0, 218.0], [], [], 0.00107, 0.07, 4822.23],     
    [[7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], [400, 800, 2200, 5200, 11000, 24200], [8.25, 9.0, 9.8, 10.2, 11.2], [], [], 0.00107, 0.07, 11088.99],   
]

# รัน unit tests
for test_case in test_data:
    test_calculate_tp_pricefibo5_cashflow(*test_case)


[PASS]: list_level_pricefibo [56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], list_level_volumefibo [100, 100, 300, 700, 1600, 3500], list_avgcost_pricefibo [61.75, 65.5, 70.0, 73.75, 78.75], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 3023.15, actual_net_cashflow: 3023.15
[PASS]: list_level_pricefibo [177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], list_level_volumefibo [100, 100, 100, 200, 500, 1200], list_avgcost_pricefibo [188.0, 192.5, 198.5, 206.0, 218.0], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 4822.23, actual_net_cashflow: 4822.23
[PASS]: list_level_pricefibo [7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], list_level_volumefibo [400, 800, 2200, 5200, 11000, 24200], list_avgcost_pricefibo [8.25, 9.0, 9.8, 10.2, 11.2], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 11088.99, actual_net_cashflow: 11088.99


## calculate_tp_pricefibo6_cashflow

In [2405]:
def calculate_tp_pricefibo6_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT):

    # หาเงินสดฝั่งขายรวม fibo level 545.5
    list_sell_midcashflow.clear()
    list_sell_midcashflow.append((list_level_pricefibo[1] * list_level_volumefibo[0]) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION) - (list_level_pricefibo[1] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[2] * list_level_volumefibo[1]) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION) - (list_level_pricefibo[2] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[3] * list_level_volumefibo[2]) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION) - (list_level_pricefibo[3] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[4] * list_level_volumefibo[3]) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION) - (list_level_pricefibo[4] * list_level_volumefibo[3] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[5] * list_level_volumefibo[4]) - (list_level_pricefibo[5] * list_level_volumefibo[4] * COMMISSION) - (list_level_pricefibo[5] * list_level_volumefibo[4] * COMMISSION * VAT))
    list_sell_midcashflow.append((list_level_pricefibo[6] * list_level_volumefibo[5]) - (list_level_pricefibo[6] * list_level_volumefibo[5] * COMMISSION) - (list_level_pricefibo[6] * list_level_volumefibo[5] * COMMISSION * VAT))

    # หาเงินสดฝั่งซื้อคืนรวม fibo level 545.5
    list_buy_midcashflow.clear()
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[0]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[0] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[0] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[1]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[1] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[1] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[2]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[2] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[2] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[3]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[3] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[3] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[4]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[4] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[4] * COMMISSION * VAT))
    list_buy_midcashflow.append((list_avgcost_pricefibo[4] * list_level_volumefibo[5]) + (list_avgcost_pricefibo[4] * list_level_volumefibo[5] * COMMISSION) + (list_avgcost_pricefibo[4] * list_level_volumefibo[5] * COMMISSION * VAT))

    # หากระแสเงินสดแฝงได้รับจาก fibo level 3545.5
    net_cashflow = round((sum(list_sell_midcashflow)- sum(list_buy_midcashflow)), 2)
    return net_cashflow


def test_calculate_tp_pricefibo6_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow = calculate_tp_pricefibo6_cashflow(list_level_pricefibo, list_level_volumefibo, list_avgcost_pricefibo, list_buy_midcashflow, list_sell_midcashflow, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_level_pricefibo {list_level_pricefibo}, list_level_volumefibo {list_level_volumefibo}, '
              f'list_avgcost_pricefibo {list_avgcost_pricefibo}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [[56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], [100, 100, 300, 700, 1600, 3500], [61.75, 65.5, 70.0, 73.75, 78.75], [], [], 0.00107, 0.07, 7603.96],  
    [[177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], [100, 100, 100, 200, 500, 1200], [188.0, 192.5, 198.5, 206.0, 218.0], [], [], 0.00107, 0.07, 10188.87],     
    [[7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], [400, 800, 2200, 5200, 11000, 24200], [8.25, 9.0, 9.8, 10.2, 11.2], [], [], 0.00107, 0.07, 24687.13],   
]

# รัน unit tests
for test_case in test_data:
    test_calculate_tp_pricefibo6_cashflow(*test_case)


[PASS]: list_level_pricefibo [56.5, 61.5, 64.5, 69.5, 74.5, 77.75, 84.25], list_level_volumefibo [100, 100, 300, 700, 1600, 3500], list_avgcost_pricefibo [61.75, 65.5, 70.0, 73.75, 78.75], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 7603.96, actual_net_cashflow: 7603.96
[PASS]: list_level_pricefibo [177.5, 187.5, 193.5, 204.0, 214.0, 220.0, 233.0], list_level_volumefibo [100, 100, 100, 200, 500, 1200], list_avgcost_pricefibo [188.0, 192.5, 198.5, 206.0, 218.0], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 10188.87, actual_net_cashflow: 10188.87
[PASS]: list_level_pricefibo [7.1, 8.1, 8.7, 9.7, 10.7, 11.3, 12.6], list_level_volumefibo [400, 800, 2200, 5200, 11000, 24200], list_avgcost_pricefibo [8.25, 9.0, 9.8, 10.2, 11.2], COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 24687.13, actual_net_cashflow: 24687.13


## update_trading_history_fibo

In [2406]:
def update_trading_history_fibo2(trade_history_df, list_buy_midcashflow, list_data_index_mid):
    
        # fibo_100
        trade_history_df.loc[list_data_index_mid[0], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[0], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[0], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[0], 'cash_buy'] = list_buy_midcashflow[0]
        trade_history_df.loc[list_data_index_mid[0], 'price_buy'] = trade_history_df.loc[list_data_index_mid[1], 'point_buyback']        
        # fibo_161.8
        trade_history_df.loc[list_data_index_mid[1], 'price_buy'] = trade_history_df.loc[list_data_index_mid[1], 'point_buyback']
        trade_history_df.loc[list_data_index_mid[1], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[1], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[1], 'cash_buy'] = list_buy_midcashflow[1]
        trade_history_df.loc[list_data_index_mid[1], 'cash'] = 0.00

def update_trading_history_fibo3(trade_history_df, list_buy_midcashflow, list_data_index_mid):
    
        # fibo_100
        trade_history_df.loc[list_data_index_mid[0], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[0], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[0], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[0], 'cash_buy'] = list_buy_midcashflow[0]
        trade_history_df.loc[list_data_index_mid[0], 'price_buy'] = trade_history_df.loc[list_data_index_mid[2], 'point_buyback']        
        # fibo_161.8
        trade_history_df.loc[list_data_index_mid[1], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[1], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[1], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[1], 'cash_buy'] = list_buy_midcashflow[1]
        trade_history_df.loc[list_data_index_mid[1], 'price_buy'] = trade_history_df.loc[list_data_index_mid[2], 'point_buyback']
        # fibo_261.8
        trade_history_df.loc[list_data_index_mid[2], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[2], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[2], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[2], 'cash_buy'] = list_buy_midcashflow[2]
        trade_history_df.loc[list_data_index_mid[2], 'price_buy'] = trade_history_df.loc[list_data_index_mid[2], 'point_buyback']

def update_trading_history_fibo4(trade_history_df, list_buy_midcashflow, list_data_index_mid):
    
        # fibo_100
        trade_history_df.loc[list_data_index_mid[0], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[0], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[0], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[0], 'cash_buy'] = list_buy_midcashflow[0]
        trade_history_df.loc[list_data_index_mid[0], 'price_buy'] = trade_history_df.loc[list_data_index_mid[3], 'point_buyback']        
        # fibo_161.8
        trade_history_df.loc[list_data_index_mid[1], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[1], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[1], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[1], 'cash_buy'] = list_buy_midcashflow[1]
        trade_history_df.loc[list_data_index_mid[1], 'price_buy'] = trade_history_df.loc[list_data_index_mid[3], 'point_buyback']
        # fibo_261.8
        trade_history_df.loc[list_data_index_mid[2], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[2], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[2], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[2], 'cash_buy'] = list_buy_midcashflow[2]
        trade_history_df.loc[list_data_index_mid[2], 'price_buy'] = trade_history_df.loc[list_data_index_mid[3], 'point_buyback']
        # fibo_361.8
        trade_history_df.loc[list_data_index_mid[3], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[3], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[3], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[3], 'cash_buy'] = list_buy_midcashflow[3]
        trade_history_df.loc[list_data_index_mid[3], 'price_buy'] = trade_history_df.loc[list_data_index_mid[3], 'point_buyback']

def update_trading_history_fibo5(trade_history_df, list_buy_midcashflow, list_data_index_mid):
    
        # fibo_100
        trade_history_df.loc[list_data_index_mid[0], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[0], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[0], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[0], 'cash_buy'] = list_buy_midcashflow[0]
        trade_history_df.loc[list_data_index_mid[0], 'price_buy'] = trade_history_df.loc[list_data_index_mid[4], 'point_buyback']        
        # fibo_161.8
        trade_history_df.loc[list_data_index_mid[1], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[1], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[1], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[1], 'cash_buy'] = list_buy_midcashflow[1]
        trade_history_df.loc[list_data_index_mid[1], 'price_buy'] = trade_history_df.loc[list_data_index_mid[4], 'point_buyback']
        # fibo_261.8
        trade_history_df.loc[list_data_index_mid[2], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[2], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[2], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[2], 'cash_buy'] = list_buy_midcashflow[2]
        trade_history_df.loc[list_data_index_mid[2], 'price_buy'] = trade_history_df.loc[list_data_index_mid[4], 'point_buyback']
        # fibo_361.8
        trade_history_df.loc[list_data_index_mid[3], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[3], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[3], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[3], 'cash_buy'] = list_buy_midcashflow[3]
        trade_history_df.loc[list_data_index_mid[3], 'price_buy'] = trade_history_df.loc[list_data_index_mid[4], 'point_buyback']
        # fibo_423.6
        trade_history_df.loc[list_data_index_mid[4], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[4], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[4], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[4], 'cash_buy'] = list_buy_midcashflow[4]
        trade_history_df.loc[list_data_index_mid[4], 'price_buy'] = trade_history_df.loc[list_data_index_mid[4], 'point_buyback']

def update_trading_history_fibo6(trade_history_df, list_buy_midcashflow, list_data_index_mid):
    
        # fibo_100
        trade_history_df.loc[list_data_index_mid[0], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[0], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[0], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[0], 'cash_buy'] = list_buy_midcashflow[0]
        trade_history_df.loc[list_data_index_mid[0], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']        
        # fibo_161.8
        trade_history_df.loc[list_data_index_mid[1], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[1], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[1], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[1], 'cash_buy'] = list_buy_midcashflow[1]
        trade_history_df.loc[list_data_index_mid[1], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']
        # fibo_261.8
        trade_history_df.loc[list_data_index_mid[2], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[2], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[2], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[2], 'cash_buy'] = list_buy_midcashflow[2]
        trade_history_df.loc[list_data_index_mid[2], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']
        # fibo_361.8
        trade_history_df.loc[list_data_index_mid[3], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[3], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[3], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[3], 'cash_buy'] = list_buy_midcashflow[3]
        trade_history_df.loc[list_data_index_mid[3], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']
        # fibo_423.6
        trade_history_df.loc[list_data_index_mid[4], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[4], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[4], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[4], 'cash_buy'] = list_buy_midcashflow[4]
        trade_history_df.loc[list_data_index_mid[4], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']
        # fibo_545.5
        trade_history_df.loc[list_data_index_mid[5], 'volume_buy'] = trade_history_df.loc[list_data_index_mid[5], 'volume_sell']
        trade_history_df.loc[list_data_index_mid[5], 'cash'] = 0.00
        trade_history_df.loc[list_data_index_mid[5], 'cash_buy'] = list_buy_midcashflow[5]
        trade_history_df.loc[list_data_index_mid[5], 'price_buy'] = trade_history_df.loc[list_data_index_mid[5], 'point_buyback']

## update_history_fibo

In [2407]:
def update_history_fibo2(df_trade_hist, var, list_data_index_mid):
    index = list_data_index_mid[0]
    df_trade_hist.loc[index, 'cash'] = var['list_buy_midcashflow'][0]
    df_trade_hist.loc[index, 'point_buyback'] = var['list_avgcost_pricefibo'][0]

def update_history_fibo3(df_trade_hist, var, list_data_index_mid):
    index = list_data_index_mid[0] 
    index2 = list_data_index_mid[1] 
    df_trade_hist.loc[index2, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index, 'cash'] = var['list_buy_midcashflow'][0]
    df_trade_hist.loc[index2, 'cash'] = var['list_buy_midcashflow'][1]
    df_trade_hist.loc[index, 'point_buyback'] = var['list_avgcost_pricefibo'][1]
    df_trade_hist.loc[index2, 'point_buyback'] = var['list_avgcost_pricefibo'][1]

def update_history_fibo4(df_trade_hist, var, list_data_index_mid):
    index = list_data_index_mid[0] 
    index2 = list_data_index_mid[1] 
    index3 = list_data_index_mid[2] 
    df_trade_hist.loc[index2, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index3, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index, 'cash'] = var['list_buy_midcashflow'][0]
    df_trade_hist.loc[index2, 'cash'] = var['list_buy_midcashflow'][1]
    df_trade_hist.loc[index3, 'cash'] = var['list_buy_midcashflow'][2]
    df_trade_hist.loc[index, 'point_buyback'] = var['list_avgcost_pricefibo'][2]
    df_trade_hist.loc[index2, 'point_buyback'] = var['list_avgcost_pricefibo'][2]
    df_trade_hist.loc[index3, 'point_buyback'] = var['list_avgcost_pricefibo'][2]

def update_history_fibo5(df_trade_hist, var, list_data_index_mid):
    index = list_data_index_mid[0] 
    index2 = list_data_index_mid[1] 
    index3 = list_data_index_mid[2] 
    index4 = list_data_index_mid[3] 
    df_trade_hist.loc[index2, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index3, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index4, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index, 'cash'] = var['list_buy_midcashflow'][0]
    df_trade_hist.loc[index2, 'cash'] = var['list_buy_midcashflow'][1]
    df_trade_hist.loc[index3, 'cash'] = var['list_buy_midcashflow'][2]
    df_trade_hist.loc[index4, 'cash'] = var['list_buy_midcashflow'][3]
    df_trade_hist.loc[index, 'point_buyback'] = var['list_avgcost_pricefibo'][3]
    df_trade_hist.loc[index2, 'point_buyback'] = var['list_avgcost_pricefibo'][3]
    df_trade_hist.loc[index3, 'point_buyback'] = var['list_avgcost_pricefibo'][3]
    df_trade_hist.loc[index4, 'point_buyback'] = var['list_avgcost_pricefibo'][3]

def update_history_fibo6(df_trade_hist, var, list_data_index_mid):
    index = list_data_index_mid[0] 
    index2 = list_data_index_mid[1] 
    index3 = list_data_index_mid[2] 
    index4 = list_data_index_mid[3] 
    index5 = list_data_index_mid[4] 
    df_trade_hist.loc[index2, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index3, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index4, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index5, 'cashflow_mid'] = 0.00
    df_trade_hist.loc[index, 'cash'] = var['list_buy_midcashflow'][0]
    df_trade_hist.loc[index2, 'cash'] = var['list_buy_midcashflow'][1]
    df_trade_hist.loc[index3, 'cash'] = var['list_buy_midcashflow'][2]
    df_trade_hist.loc[index4, 'cash'] = var['list_buy_midcashflow'][3]
    df_trade_hist.loc[index5, 'cash'] = var['list_buy_midcashflow'][4]
    df_trade_hist.loc[index, 'point_buyback'] = var['list_avgcost_pricefibo'][4]
    df_trade_hist.loc[index2, 'point_buyback'] = var['list_avgcost_pricefibo'][4]
    df_trade_hist.loc[index3, 'point_buyback'] = var['list_avgcost_pricefibo'][4]
    df_trade_hist.loc[index4, 'point_buyback'] = var['list_avgcost_pricefibo'][4]
    df_trade_hist.loc[index5, 'point_buyback'] = var['list_avgcost_pricefibo'][4]

## calculate_front_cashflow

In [2408]:
def calculate_front_cashflow(list_sell_frontprice, list_sell_frontvolume, point_buyback, COMMISSION, VAT):

    # หากระแสเงินสดแฝงกองหน้า
    sell_cashflow = (list_sell_frontprice * list_sell_frontvolume) - (list_sell_frontprice * list_sell_frontvolume * COMMISSION) - (list_sell_frontprice * list_sell_frontvolume * COMMISSION * VAT)
    buy_cashflow = (point_buyback * list_sell_frontvolume) + (point_buyback * list_sell_frontvolume * COMMISSION) + (point_buyback * list_sell_frontvolume * COMMISSION * VAT)
    net_cashflow = round((sell_cashflow - buy_cashflow), 2)
    return net_cashflow, sell_cashflow, buy_cashflow, point_buyback

def test_calculate_front_cashflow(list_sell_frontprice, list_sell_frontvolume, point_buyback, COMMISSION, VAT, expected_net_cashflow):
    actual_net_cashflow, actual_sell_cashflow, actual_buy_cashflow, actual_point_buyback = calculate_front_cashflow(list_sell_frontprice, list_sell_frontvolume, point_buyback, COMMISSION, VAT)
    
    if actual_net_cashflow == expected_net_cashflow:
        print(f'[PASS]: list_sell_frontprice {list_sell_frontprice}, list_sell_frontvolume {list_sell_frontvolume}, '
              f'point_buyback {point_buyback}, COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')
    else:
        print(f'[FAILED]: list_sell_frontprice {list_sell_frontprice}, list_sell_frontvolume {list_sell_frontvolume}, '
              f'point_buyback {point_buyback},  COMMISSION {COMMISSION}, VAT {VAT} \n'
              f'       expected_net_cashflow: {expected_net_cashflow}, actual_net_cashflow: {actual_net_cashflow}')

# ตัวอย่างข้อมูลทดสอบ
test_data = [
    [56.25, 100,  23.5,  0.00107, 0.07, 3265.87],  
    [30, 400, 20.0, 0.00107, 0.07, 3977.10],  
]

# รัน unit tests
for test_case in test_data:
    test_calculate_front_cashflow(*test_case)


[PASS]: list_sell_frontprice 56.25, list_sell_frontvolume 100, point_buyback 23.5, COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 3265.87, actual_net_cashflow: 3265.87
[PASS]: list_sell_frontprice 30, list_sell_frontvolume 400, point_buyback 20.0, COMMISSION 0.00107, VAT 0.07 
       expected_net_cashflow: 3977.1, actual_net_cashflow: 3977.1


## update_trading_history_front2

In [2409]:
def update_trading_history_front2(trade_history_df, list_data_index_front):
    
    for index in list_data_index_front:
        trade_history_df.loc[index, 'volume_buy'] = trade_history_df.loc[index, 'volume_sell']
        trade_history_df.loc[index, 'cash_buy'] = trade_history_df.loc[index, 'cash']
        trade_history_df.loc[index, 'cash'] = 0.00
        trade_history_df.loc[index, 'price_buy'] = trade_history_df.loc[index, 'point_buyback']


## update_sum_port

In [2410]:
def update_sum_port(date, sum_volume, increase_volume, net_volume, cashflow_back, cashflow_mid, cashflow_front, cashflow_convert, cashflow_short, cash_remaining, cashflow_buymore, cashflow_reserve, cashflow_fee, net_cash, portfolio_value, nav , benchmark):
    new_data = {
                "date": date ,
                "sum_volume": sum_volume,
                "increase_volume": increase_volume,
                "net_volume": net_volume,
                "cashflow_back": cashflow_back,
                "cashflow_mid": cashflow_mid,
                "cashflow_front": cashflow_front,
                "cashflow_convert": cashflow_convert,
                "cashflow_short": cashflow_short,
                "cash_buyback": cash_remaining,
                "cashflow_buymore": cashflow_buymore,
                "cashflow_reserve": cashflow_reserve,
                "cashflow_fee": cashflow_fee,
                "total_cash": net_cash,
                "total_equity": portfolio_value,
                "nav": nav,
                "benchmark": benchmark
                }
    return pd.DataFrame([new_data])


## calculate_range_of_stock

In [2411]:
def calculate_range_of_stock(start_price, end_price):
    current_price = start_price
    number_of_spreads = 0
    if current_price > 0:
        while current_price < end_price:
            spread = calculate_spread(current_price)
            next_price = current_price + spread

            # ตรวจสอบว่า next_price จะข้ามเข้าสู่ช่วงราคาใหม่หรือไม่
            if calculate_spread(next_price) != spread and next_price > current_price:
                # คำนวณจำนวนช่องในช่วงราคาปัจจุบันก่อนเปลี่ยนช่วงราคา
                number_of_spreads += (min(next_price, end_price) - current_price) / spread
                current_price = next_price
                continue

            # จำกัดให้ไม่เกิน end_price
            if next_price > end_price:
                number_of_spreads += (end_price - current_price) / spread
                break

            current_price = next_price
            number_of_spreads += 1

    return round(number_of_spreads)

def test_calculate_range_of_stock(start_price, end_price, expected_output):
    actual_output = calculate_range_of_stock(start_price, end_price)
    if actual_output == expected_output:
        print('[PASS]: start_price {},end_price {}, expect: {}, actual: {}'.format(start_price, end_price, expected_output, actual_output))
    else:
        print('[FAILED]: start_price {},end_price {}, expect: {}, actual: {}'.format(start_price, end_price, expected_output, actual_output))

test_data = [
    [1.99, 2.02, 2],
    [4.98, 5.05, 2],
    [9.95, 10.10, 2],
    [24.90, 25.25, 2],
    [99.75, 100.50, 2],
    [199.50, 201.00, 2],
    [399, 402, 2],
    [2, 2.4, 20],
    [10, 12, 20],
    [100, 160, 120],
    [160, 100, 0],
    [12, 10, 0],
    [0, 10, 0],

]

for i in range(0,len(test_data),1):  
    test_calculate_range_of_stock(test_data[i][0],test_data[i][1],test_data[i][2] )

[PASS]: start_price 1.99,end_price 2.02, expect: 2, actual: 2
[PASS]: start_price 4.98,end_price 5.05, expect: 2, actual: 2
[PASS]: start_price 9.95,end_price 10.1, expect: 2, actual: 2
[PASS]: start_price 24.9,end_price 25.25, expect: 2, actual: 2
[PASS]: start_price 99.75,end_price 100.5, expect: 2, actual: 2
[PASS]: start_price 199.5,end_price 201.0, expect: 2, actual: 2
[PASS]: start_price 399,end_price 402, expect: 2, actual: 2
[PASS]: start_price 2,end_price 2.4, expect: 20, actual: 20
[PASS]: start_price 10,end_price 12, expect: 20, actual: 20
[PASS]: start_price 100,end_price 160, expect: 120, actual: 120
[PASS]: start_price 160,end_price 100, expect: 0, actual: 0
[PASS]: start_price 12,end_price 10, expect: 0, actual: 0
[PASS]: start_price 0,end_price 10, expect: 0, actual: 0


## update_increase_volume

In [2412]:
def update_increase_volume(price, cashflow_buymore, COMMISSION, VAT):
    find_increase_volume = calculate_floor_stock(cashflow_buymore / price)
    net_buy_cashflow = (price * find_increase_volume) + (price * find_increase_volume * COMMISSION) + (price * find_increase_volume * COMMISSION * VAT)

    # ลดจำนวนหุ้นทีละ 100 จนกว่าเงินที่ใช้จะไม่เกินงบประมาณ
    while (cashflow_buymore - net_buy_cashflow) < 0 and find_increase_volume >= 100:
        find_increase_volume -= 100
        net_buy_cashflow = (price * find_increase_volume) + (price * find_increase_volume * COMMISSION) + (price * find_increase_volume * COMMISSION * VAT)
    
    return find_increase_volume, net_buy_cashflow



## update_trading_backconvert

In [2413]:
def update_trading_backconvert(list_data_index_back, trade_history_df, stock ,stock_vars, send_stock):
    # ปรับ update_trading_history_back ใหม่
    for index in list_data_index_back: 
        trade_history_df.loc[index, 'cash'] = 0.00
        trade_history_df.loc[index, 'type'] = 'BTS CV send {}'.format(send_stock)
    list_data_index_back.clear() 
    stock_vars['list_sell_backprice'].clear()
    stock_vars['list_sell_backvolume'].clear()
    stock_vars['count_back'] = 0

## update_trading_frontconvert

In [2414]:
def update_trading_frontconvert(list_data_index_front, trade_history_df, stock ,stock_vars, send_stock):
    # ปรับ update_trading_history_front ใหม่
    for index in list_data_index_front: 
        trade_history_df.loc[index, 'cash'] = 0.00
        trade_history_df.loc[index, 'type'] = 'FTS CV send {}'.format(send_stock)
    list_data_index_front.clear() 
    stock_vars['list_sell_frontprice'].clear()
    stock_vars['list_sell_frontvolume'].clear()
    stock_vars['list_data_index_front'].clear()


# Ontick

In [2415]:
# BTS Back Trading Strategy (กองหลัง)
# FIBO Front Trading Strategy (กองกลาง)
# FTS Short Trading Strategy (กองหน้า)
# CV Convert Trading Strategy (แปลงร่าง)
# STS Short Trading Strategy (กองหน้า)



# ตัวแปรใช้ร่วมกัน
TOTAL_BUDGET = 0
COMMISSION = 0.00107
VAT = 0.07
PERCENTAGE_BACK = 0.10
PERCENTAGE_FIBO = [0.0125, 0.0251, 0.0502, 0.1003, 0.2006, 0.4013] 
PERCENTAGE_FRONT = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
SUM_MID_PERCENTAGE = 0.79
SUM_FRONT_PERCENTAGE = 0.21
SUM_ฺBACK_PERCENTAGE = 0.30
PERCENTAGE_BUYMORE = 0.70
PERCENTAGE_RESERVE = 0.20
PERCENTAGE_FEE = 0.10


# ใช้บันทึกเก็บกระแสเงินสด
percentages = {
    'buymore': PERCENTAGE_BUYMORE,
    'reserve': PERCENTAGE_RESERVE,
    'fee': PERCENTAGE_FEE
}

# list รายชื่อหุ้น
list_stock = ['SCC','ADVANC','KBANK','KCE','EA','CBG','DELTA','INTUCH','BH','BBL','SCGP','SAWAD','KTC','PTTEP','GULF','TOP','EGCO','MTC','GPSC','TISCO','BGRIM','PTTGC','AOT','CPALL','CPN','RATCH','TU','IVL','CENTEL','OR','CRC','MINT','PTT','CPF','OSP','BANPU','GLOBAL','BDMS','KTB','True','HMPRO','BEM','AWC','LH','BTS','WHA','TTB']

# COM7 ข้อมูล error และ SCB,TLI ข้อมูลไม่ครบ
#list_stock = ['SCC', 'ADVANC', 'KBANK', 'KCE', 'EA', 'CBG', 'DELTA', 'INTUCH', 'BH', 'BBL']

# list อันดับหุ้นที่มี std สูงสุด 10 อันดับ
list_top10_std = ['SCC', 'ADVANC', 'KBANK', 'KCE', 'EA', 'CBG', 'DELTA', 'INTUCH', 'BH', 'BBL']

# โหลดข้อมูลวันทั้งหมด
data_date = pd.read_excel(r'C:\Users\ming\Desktop\MyJob\dsm\Date_time.xlsx')

# สร้าง dict เก็บตัวแปรแยกหุ้นแต่ละตัวกัน
for stock in list_stock:

    # dataframe ประวัติการซื้อขาย
    df_trade_hist[stock] = deepcopy(trade_hist)

    # dataframe ข้อมูลราคา
    var[stock] = deepcopy(dict_variable)
    
    # dataframe ของหุ้นแต่ละตัว
    url = r'C:\Users\ming\Desktop\MyJob\DSM1.1.2\data\set50.xlsx'
    df[stock]  = pd.read_excel(url, sheet_name=stock)

# ดึงข้อมูลวันมาทีละ 1 วัน
for date in data_date['Date_time']:

    # reset ค่า dict_keep_variable ของ df_dt
    dict_keep_variable = {
                "date": object ,
                "sum_volume": 0,
                "increase_volume": 0,
                "net_volume": 0,
                "sum_cashflow_back": 0,
                "sum_cashflow_mid": 0,
                "sum_cashflow_front": 0,
                "sum_cashflow_convert": 0,
                "sum_cashflow_short": 0,
                "cash_buyback": 0.00,
                "cashflow_buymore": 0.00,
                "cashflow_reserve": 0.00,
                "cashflow_fee": 0.00,
                "total_cash": 0.00,
                "total_equity": 0.00,
                "nav": 0.00,
                "benchmark": 0.00                    
            }
    
    # โหลดข้อมูลย้อนหลังของหุ้นแต่ละตัวในวันที่กำหนด
    for stock in list_stock :

        # โหลดข้อมูลราคาปิดรายวัน
        for price in df[stock][df[stock].date == date]['Close']:
            
            # ปรับ spread ให้ใกล้เคียง
            price = round_spread(price)

#--------------------------------------------------------------------------------------------- เริ่มซื้อครั้งแรก ---------------------------------------------------------------------------------------------#
           
            
            # จุดหาราคาและจำนวนหุ้นทั้งหมดครั้งแรก
            if var[stock]['start_in_tradezone'] == False and var[stock]['count_index'] < len(df[stock]):
                if price >= df[stock]['mid_band'].iloc[var[stock]['count_index']] and price <= df[stock]['up_band'].iloc[var[stock]['count_index']] :
                    var[stock]['start_in_tradezone'] = True
                    var[stock]['FIRST_PRICE'] = round_spread(df[stock]['mid_band'].iloc[var[stock]['count_index']])

                    if stock in list_top10_std:
                        var[stock]['budget'] = 2000000 # top 10 ให้ทุน 60m
                    else:
                        var[stock]['budget'] = 2000000 # ไม่ติด top 10 ให้ทุน 20m

                    var[stock]['FIRST_VOLUME'] = round(calculate_floor_stock(var[stock]['budget']/var[stock]['FIRST_PRICE']))
                    TOTAL_BUDGET += var[stock]['budget']
                    var[stock]['sum_volume'] += var[stock]['FIRST_VOLUME']

            # เริ่มเทรด 
            if var[stock]['start_in_tradezone'] and var[stock]['count_index'] < len(df[stock]):

#--------------------------------------------------------------------------------------------- เริ่มหาต้นทุนเฉลี่ย ---------------------------------------------------------------------------------------------#
  
                # หาราคาต้นทุนเฉลี่ยของพอร์ต
                keep_sum_value = 0
                keep_sum_volume = 0

                for i in range (0, len(df_trade_hist[stock]), 1):
                    if df_trade_hist[stock].loc[i, 'price_buy'] > 0 and  df_trade_hist[stock].loc[i, 'volume_buy'] > 0:                      
                        keep_sum_value += (df_trade_hist[stock].loc[i, 'price_buy'] * df_trade_hist[stock].loc[i, 'volume_buy'])
                        keep_sum_volume += df_trade_hist[stock].loc[i, 'volume_buy']
              
                if keep_sum_volume > 0:
                    var[stock]['avg_price'] = round_spread(keep_sum_value / keep_sum_volume)
                else:
                    var[stock]['avg_price'] = var[stock]['FIRST_PRICE']  
#--------------------------------------------------------------------------------------------- หาจุดต่ำสุดสูงสุด ---------------------------------------------------------------------------------------------#

                # หาจุดที่ราคาต่ำที่สุด
                if var[stock]['start_in_backzone']:
                    if len(var[stock]['list_sell_backprice']) > 0:
                        var[stock]['lower_price'] = max(var[stock]['list_sell_backprice'])
                else:
                    var[stock]['lower_price'] = var[stock]['FIRST_PRICE']                

                # หาจุดที่ราสูงที่สุด
                var[stock]['current_price'] = price
                var[stock]['test_max'] = max(price, var[stock]['test_max'])
                var[stock]['test_min'] = min(price, var[stock]['test_min'])

#--------------------------------------------------------------------------------------------- เริ่มกองหลัง ---------------------------------------------------------------------------------------------#
                
                # ปรับกองหลังต้องชนเส้น bb lower ก่อนถึงจะเริ่มขาย
                if price <= df[stock]['low_band'].iloc[var[stock]['count_index']] and price < var[stock]['FIRST_PRICE'] and var[stock]['start_in_backzone'] == False:
                    var[stock]['signal_backprice'] = price
                    var[stock]['start_in_backzone'] = True   
                    
                # เริ่มขายกองหลัง
                if price <= var[stock]['signal_backprice'] and var[stock]['start_in_backzone'] :

                    ## เกิดกองหลังสามารถส่งกองหลังแปลงร่างได้ใหม่หากราคาขึ้นไป 50 ช่องอีกครั้ง
                    var[stock]['start_send_convert'] = False

                    ## บันทึกข้อมูลระยะของราคาปิดที่ต่ำลงมาจาก signal_backprice
                    var[stock]['list_range_backprice'], signal, var[stock]['count_range_backprice'] = calculate_range_price(price, var[stock]['signal_backprice'], var[stock]['count_range_backprice'])

                    ## เอาราคาใน list_range_backprice มาเข้าเงื่อนไข
                    for i in var[stock]['list_range_backprice']:
                        var[stock]['count_range_backprice'] -= 1           

                        ### เช็คทุกราคาในลิสจบ Loop กองหลัง
                        if var[stock]['count_range_backprice'] == 0:
                            var[stock]['list_range_backprice'].clear() 

                        ### เช็คจุดซื้อคืนกองหลังและลบราคาที่ซื้อคืนได้
                        if var[stock]['count_back'] == 3:
                            if price <= calculate_buyback_point(var[stock]['list_sell_backprice'][0], 5, -1):
                                var[stock]['list_sell_backprice'].pop(0)
                                var[stock]['list_sell_backvolume'].pop(0)
                                var[stock]['count_back'] -= 1

                                #### เก็บประวัติการซื้อขายกองหลังฝั่งซื้อ
                                update_trading_history_back(df_trade_hist[stock], var[stock], var[stock]['list_data_index_back'][0])

                        ### เช็คว่าถึงจุดที่เข้าเงื่อนไขขายกองหลังตัวต่อไปหรือไม่
                        if var[stock]['count_back'] < 3 and i not in var[stock]['list_history_sell_backprice'] :

                            #### บันทึกข้อมูลขายกองหลัง
                            var[stock]['list_history_sell_backprice'].append(i)
                            var[stock]['list_sell_backprice'].append(i)
                            if var[stock]['count_slzone'] == 0:
                                var[stock]['list_sell_backvolume'].append(round(calculate_floor_stock(var[stock]['sum_volume'] * PERCENTAGE_BACK)))
                            else:
                                var[stock]['list_sell_backvolume'].append(var[stock]['previous_backvolume'])                           
                            var[stock]['signal_backprice'] = calculate_buyback_point(i, 2, -1)
                            var[stock]['count_back'] += 1
                            var[stock]['stat_count_back'] +=1
                            var[stock]['count_sl_convert'] +=1


                            #### จัดการกระแสเงินสด
                            keep_cashflow, keep_volume_sell, keep_cash_sell, keep_cash_buy, keep_point_buyback = calculate_cashflow(i, var[stock]['list_sell_backvolume'][-1], COMMISSION, VAT, 5)
                            update_cashflows(var[stock], keep_cashflow, percentages, 'กองหลัง')
                                                    
                            #### เก็บประวัติการซื้อขายกองหลังฝั่งขาย
                            var[stock]['list_data_index_back'].append(var[stock]['data_index'])
                            var[stock]['data_index'] += 1 
                            var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                            new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'BTS', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], i, keep_volume_sell, 0.00, 0, keep_point_buyback, keep_cash_sell, 0.00, keep_cash_buy, keep_cashflow, 0.00, 0.00, 0.00,0.00, var[stock]['cashflow_buymore'])
                            df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)


                            # เช็คจำนวนกองหลังเกิดขึ้นครบ 25 ครั้งหรือไม่ถ้าครบจะทำการแปลงร่างและเช็คว่าไม่มีกองกลางขายอยู่
                            if  var[stock]['count_slzone'] == 0 and var[stock]['count_sl_convert'] >= 25 and var[stock]['count_fibo'] == 0 and var[stock]['count_short'] == 0  : # หรืออาจเปลี่ยนเป็นกองหลังลงมากกว่า 50 ช่องจากราคาเริ่มต้นลองทดสอบแบบไหนได้กสงฝ. มากกว่ากัน
                                var[stock]['lower_price_convert'] = i

                                # จำนวนหุ้นทั้งหมดที่เหลืออยู่ยกเว้นกองหลังเพื่อขาย ณ ราคาปัจจุบัน
                                var[stock]['volume_slconvert'] = (var[stock]['sum_volume'] - (sum(var[stock]['list_sell_backvolume']) + sum(var[stock]['list_sell_midvolume']) + sum(var[stock]['list_sell_frontvolume']) + sum(var[stock]['list_sell_shortvolume'])))

                                # หาผลรวมของเงินสดที่ซื้อคืนไม่ได้ของกองหลังทั้งหมดและเพิ่มเข้าไปในหุ้นที่จะแปลงร่าง
                                var[stock]['cashflow_slconvert'] = (var[stock]['lower_price_convert']  * var[stock]['volume_slconvert']) - (var[stock]['lower_price_convert']  * var[stock]['volume_slconvert'] * COMMISSION) - (var[stock]['lower_price_convert']  * var[stock]['volume_slconvert'] * COMMISSION * VAT)

                                # ปรับเงินลงทุนของหุ้นเดิม
                                var[stock]['budget'] -= var[stock]['cashflow_slconvert']   
                                var[stock]['previous_backvolume'] = var[stock]['list_sell_backvolume'][-1]                  
                                var[stock]['count_slzone'] = 1

                                ## เก็บประวัติฝั่งขายหลังแปลงร่างออกไป 
                                var[stock]['list_data_index_sl'].append(var[stock]['data_index'])
                                var[stock]['data_index'] += 1 
                                var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                                new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'],  'SL', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['lower_price_convert'] , var[stock]['volume_slconvert'], 0.00, 0, 0.00, var[stock]['cashflow_slconvert'], 0.00, var[stock]['cashflow_slconvert'], 0.00, 0.00, 0.00, 0.00,0.00, var[stock]['cashflow_buymore'])
                                df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                                                

#--------------------------------------------------------------------------------------------- เริ่มกองกลาง ---------------------------------------------------------------------------------------------#
                
                ## ปรับสัดส่วนจำนวนที่ขายกองหน้าและกองกลางตามจำนวนที่เหลือกองหลังที่ยังไม่ได้ขาย
                check_net_sell_volume = var[stock]['sum_volume'] - sum(var[stock]['list_sell_backvolume']) - sum(var[stock]['list_sell_frontvolume'])

                # จุดเข้าแรก ณ  fibo level 100
                if var[stock]['count_fibo'] == 0 and price >= df[stock]['up_band'].iloc[var[stock]['count_index']] and var[stock]['start_in_shortzone'] == False and var[stock]['count_slzone'] == 0 and check_net_sell_volume >= 3000:

                    ## ปรับสัดส่วนจำนวนที่ขายกองหน้าและกองกลางตามจำนวนที่เหลือกองหลังที่ยังไม่ได้ขาย
                    var[stock]['net_sell_midvolume'] = var[stock]['sum_volume'] - sum(var[stock]['list_sell_backvolume']) - sum(var[stock]['list_sell_frontvolume'])

                    ## หาจำนวนหุ้นที่ขายแต่ละ level fibo ของกองกลาง
                    for i in PERCENTAGE_FIBO :
                        var[stock]['list_level_volumefibo'].append(calculate_min_sellvolume(var[stock]['net_sell_midvolume'], var[stock]['list_sell_midvolume'], i, SUM_MID_PERCENTAGE))      
                        
                    ## หาจำนวนหุ้นที่ขายแต่ละ level fibo ของกองหน้า
                    var[stock]['list_level_volumefront'] = calculate_max_volumesell(var[stock]['net_sell_midvolume'], PERCENTAGE_FRONT, SUM_FRONT_PERCENTAGE )

                    var[stock]['list_sell_midprice'].append(price) 
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][0])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1

                    ## draw price fibo ทุก level
                    var[stock]['list_level_pricefibo'].append(calculate_buyback_point(price, 20, -1))  # fibo 0
                    var[stock]['list_level_pricefibo'].append(price) # fibo 100
                    draw_setting_pricefibo(var[stock]['list_level_pricefibo'])           

                    ## จุดสัญญาณระบบ short ต้องมีราคาสูงกว่าจุดต่ำสุดของกองหลังตัวบนหรือราคาต้นทุน
                    if var[stock]['list_level_pricefibo'][0] > var[stock]['lower_price']:
                        var[stock]['signal_shortprice'] = var[stock]['list_level_pricefibo'][0]

                    ## หา avgcost แต่ละ level fibo
                    calculate_avgcost_pricefibo(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'])

                    ## เก็บประวัติกองหน้า
                    find_sell_cash = (var[stock]['list_sell_midprice'][0] * var[stock]['list_sell_midvolume'][0]) - (var[stock]['list_sell_midprice'][0] * var[stock]['list_sell_midvolume'][0]  * COMMISSION) - (var[stock]['list_sell_midprice'][0] * var[stock]['list_sell_midvolume'][0]  * COMMISSION * VAT)
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO1', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][0], var[stock]['list_sell_midvolume'][0], 0.00, 0, 0.00, find_sell_cash, 0.00, find_sell_cash, 0.00, 0.00, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 100 ซื้อคืนเมื่อราคากลับตัวไป mid_band    
                if var[stock]['count_fibo'] == 1 and price <= df[stock]['mid_band'].iloc[var[stock]['count_index']] and var[stock]['signal_buyback_midband'] == False :

                    ## เช็คว่า mid_band มีระยะกำไรมากกว่า 5 ช่องหรือไม่
                    if df[stock]['mid_band'].iloc[var[stock]['count_index']] <= calculate_buyback_point(var[stock]['list_sell_midprice'][0], 5, -1)  :
                        var[stock]['list_avgcost_pricefibo'].append(round_spread(df[stock]['mid_band'].iloc[var[stock]['count_index']]))
                        var[stock]['signal_buyback_midband'] = True
                    
                    ## เปลี่ยนเงื่อนไขเมื่อซื้อคืน mid_band ไม่ได้เป็นปิดเมื่อราคาซื้อคืนมากกว่าหรือเท่ากับ 5 ช่อง
                    elif price <= calculate_buyback_point(var[stock]['list_sell_midprice'][0], 5, -1)  :
                        var[stock]['list_avgcost_pricefibo'].append(calculate_buyback_point(var[stock]['list_sell_midprice'][0], 5, -1))
                        var[stock]['signal_buyback_midband'] = True
                        
                if var[stock]['signal_buyback_midband'] :
                    

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][-1], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
        

                    ## จัดการกระแสเงินสด
                    keep_cashflow, keep_sell_cashflow, keep_buy_cashflow = calculate_midband_cashflow(var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_avgcost_pricefibo'], COMMISSION, VAT)
                    update_cashflows(var[stock], keep_cashflow, percentages, 'กองกลาง')
                    var[stock]['signal_buyback_midband'] = False
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0  
                                    
                    ## ประวัติการซื้อขายกองหน้าซื้อคืนได้
                    index = var[stock]['list_data_index_mid'][0] 
                    df_trade_hist[stock].loc[index, 'price_buy'] = var[stock]['list_avgcost_pricefibo'][-1]
                    df_trade_hist[stock].loc[index, 'volume_buy'] = df_trade_hist[stock].loc[index, 'volume_sell']
                    df_trade_hist[stock].loc[index, 'cash_buy'] = keep_buy_cashflow
                    df_trade_hist[stock].loc[index, 'cashflow_mid'] = keep_cashflow
                    df_trade_hist[stock].loc[index, 'cash'] = 0.00
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])


                # จุดเข้า ณ ราคา fibo 161.8
                if var[stock]['count_fibo'] == 1 and price >= var[stock]['list_level_pricefibo'][2]:
                    var[stock]['list_sell_midprice'].append( var[stock]['list_level_pricefibo'][2])
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][1])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1


                    ## จัดการกระแสเงินสด
                    keep_cashflow = calculate_tp_pricefibo2_cashflow(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_sell_midcashflow'],   COMMISSION, VAT)
                    var[stock]['list_dif_midcashflow'].append(keep_cashflow)
                    update_cashflows(var[stock], var[stock]['list_dif_midcashflow'][0], percentages, 'กองกลาง')

                    ## เก็บประวัติการซื้อขายกองกลางฝั่งขาย
                    update_history_fibo2(df_trade_hist[stock], var[stock], var[stock]['list_data_index_mid'])
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO2', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][1], var[stock]['list_sell_midvolume'][1], 0.00, 0, var[stock]['list_avgcost_pricefibo'][0], var[stock]['list_sell_midcashflow'][1], 0.00, var[stock]['list_buy_midcashflow'][1], 0.00, keep_cashflow, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 161.8 ถึงจุดซื้อคืน  
                if var[stock]['count_fibo'] == 2 and price <= var[stock]['list_avgcost_pricefibo'][0] :
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][0], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
        
                    ## อัพเดตประวัติการซื้อขายกองกลางซื้อคืนได้ fibo level 161.8
                    update_trading_history_fibo2(df_trade_hist[stock], var[stock]['list_buy_midcashflow'], var[stock]['list_data_index_mid'])
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])

                # จุดเข้า ณ ราคา fibo 261.8
                if var[stock]['count_fibo'] == 2 and price >= var[stock]['list_level_pricefibo'][3]:
                    var[stock]['list_sell_midprice'].append( var[stock]['list_level_pricefibo'][3])
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][2])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1

                    ## จัดการกระแสเงินสด
                    keep_cashflow = calculate_tp_pricefibo3_cashflow(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_sell_midcashflow'],   COMMISSION, VAT)
                    var[stock]['list_dif_midcashflow'].append(keep_cashflow - sum(var[stock]['list_dif_midcashflow']))
                    update_cashflows(var[stock], var[stock]['list_dif_midcashflow'][1], percentages, 'กองกลาง')

                    ## เก็บประวัติการซื้อขายกองกลางฝั่งขาย
                    update_history_fibo3(df_trade_hist[stock], var[stock], var[stock]['list_data_index_mid'])
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO3', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][2], var[stock]['list_sell_midvolume'][2], 0.00, 0, var[stock]['list_avgcost_pricefibo'][1], var[stock]['list_sell_midcashflow'][2], 0.00, var[stock]['list_buy_midcashflow'][2], 0.00, keep_cashflow, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 261.8 ถึงจุดซื้อคืน  
                if var[stock]['count_fibo'] == 3 and price <= var[stock]['list_avgcost_pricefibo'][1] :
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][1], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
        

                    ## อัพเดตประวัติการซื้อขายกองกลางซื้อคืนได้ fibo level 261.8
                    update_trading_history_fibo3(df_trade_hist[stock], var[stock]['list_buy_midcashflow'], var[stock]['list_data_index_mid'])
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])

                # จุดเข้า ณ ราคา fibo 361.8
                if var[stock]['count_fibo'] == 3 and price >= var[stock]['list_level_pricefibo'][4]:
                    var[stock]['list_sell_midprice'].append( var[stock]['list_level_pricefibo'][4])
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][3])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1

                    ## จัดการกระแสเงินสด
                    keep_cashflow = calculate_tp_pricefibo4_cashflow(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_sell_midcashflow'],   COMMISSION, VAT)
                    var[stock]['list_dif_midcashflow'].append(keep_cashflow - sum(var[stock]['list_dif_midcashflow']))
                    update_cashflows(var[stock], var[stock]['list_dif_midcashflow'][2], percentages, 'กองกลาง')

                    ## เก็บประวัติการซื้อขายกองกลางฝั่งขาย
                    update_history_fibo4(df_trade_hist[stock], var[stock], var[stock]['list_data_index_mid'])
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO4', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][3], var[stock]['list_sell_midvolume'][3], 0.00, 0, var[stock]['list_avgcost_pricefibo'][2], var[stock]['list_sell_midcashflow'][3], 0.00, var[stock]['list_buy_midcashflow'][3], 0.00, keep_cashflow, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 361.8 ถึงจุดซื้อคืน  
                if var[stock]['count_fibo'] == 4 and price <= var[stock]['list_avgcost_pricefibo'][2] and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][2], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
        
                    ## อัพเดตประวัติการซื้อขายกองกลางซื้อคืนได้ fibo level 361.8
                    update_trading_history_fibo4(df_trade_hist[stock], var[stock]['list_buy_midcashflow'], var[stock]['list_data_index_mid'])
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])

                # จุดเข้า ณ ราคา fibo 423.6
                if var[stock]['count_fibo'] == 4 and price >= var[stock]['list_level_pricefibo'][5]:
                    var[stock]['list_sell_midprice'].append( var[stock]['list_level_pricefibo'][5])
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][4])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1

                    ## จัดการกระแสเงินสด
                    keep_cashflow = calculate_tp_pricefibo5_cashflow(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_sell_midcashflow'],   COMMISSION, VAT)
                    var[stock]['list_dif_midcashflow'].append(keep_cashflow - sum(var[stock]['list_dif_midcashflow']))
                    update_cashflows(var[stock], var[stock]['list_dif_midcashflow'][3], percentages, 'กองกลาง')

                    ## เก็บประวัติการซื้อขายกองกลางฝั่งขาย
                    update_history_fibo5(df_trade_hist[stock], var[stock], var[stock]['list_data_index_mid'])
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO5', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][4], var[stock]['list_sell_midvolume'][4], 0.00, 0, var[stock]['list_avgcost_pricefibo'][3], var[stock]['list_sell_midcashflow'][4], 0.00, var[stock]['list_buy_midcashflow'][4], 0.00, keep_cashflow, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 423.6 ถึงจุดซื้อคืน  
                if var[stock]['count_fibo'] == 5 and price <= var[stock]['list_avgcost_pricefibo'][3] :
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][3], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
        
                    ## อัพเดตประวัติการซื้อขายกองกลางซื้อคืนได้ fibo level 423.6
                    update_trading_history_fibo5(df_trade_hist[stock], var[stock]['list_buy_midcashflow'], var[stock]['list_data_index_mid'])
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])

                # จุดเข้า ณ ราคา fibo 545.5
                if var[stock]['count_fibo'] == 5 and price >= var[stock]['list_level_pricefibo'][6]:
                    var[stock]['list_sell_midprice'].append( var[stock]['list_level_pricefibo'][6])
                    var[stock]['list_sell_midvolume'].append(var[stock]['list_level_volumefibo'][5])
                    var[stock]['count_fibo'] += 1
                    var[stock]['stat_count_mid'] += 1

                    ## จัดการกระแสเงินสด
                    keep_cashflow = calculate_tp_pricefibo6_cashflow(var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_sell_midcashflow'],   COMMISSION, VAT)
                    var[stock]['list_dif_midcashflow'].append(keep_cashflow - sum(var[stock]['list_dif_midcashflow']))
                    update_cashflows(var[stock], var[stock]['list_dif_midcashflow'][4], percentages, 'กองกลาง')

                    ## เก็บประวัติการซื้อขายกองกลางฝั่งขาย
                    update_history_fibo6(df_trade_hist[stock], var[stock], var[stock]['list_data_index_mid'])
                    var[stock]['list_data_index_mid'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FIBO6', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_midprice'][5], var[stock]['list_sell_midvolume'][5], 0.00, 0, var[stock]['list_avgcost_pricefibo'][4], var[stock]['list_sell_midcashflow'][5], 0.00, var[stock]['list_buy_midcashflow'][5], 0.00, keep_cashflow, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # เงื่อนไข fibo level 545.5 ถึงจุดซื้อคืน  
                if var[stock]['count_fibo'] == 6 and price <= var[stock]['list_avgcost_pricefibo'][4] :
                    var[stock]['count_fibo'] = 0
                    var[stock]['count_front'] = 0

                    # ซื้อเพิ่มจุดเดียวกับซื้อคืนกองกลาง
                    if var[stock]['cashflow_buymore'] > 0 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50:
                        keep_find_increase_volume, keep_net_buy_cashflow = update_increase_volume(var[stock]['list_avgcost_pricefibo'][4], var[stock]['cashflow_buymore'], COMMISSION, VAT)
                        var[stock]['increase_volume'] += keep_find_increase_volume
                        var[stock]['sum_volume'] += keep_find_increase_volume
                        var[stock]['cashflow_buymore'] -= keep_net_buy_cashflow
                    elif var[stock]['cashflow_buymore'] > 0:
                        dict_all_sum_port['cashflow_buymore'] += var[stock]['cashflow_buymore']
                        var[stock]['cashflow_buymore'] = 0
                        

                    ## อัพเดตประวัติการซื้อขายกองกลางซื้อคืนได้ fibo level 545.5
                    update_trading_history_fibo6(df_trade_hist[stock], var[stock]['list_buy_midcashflow'], var[stock]['list_data_index_mid'])
                    update_clear_datamid(var[stock]['list_data_index_mid'], var[stock]['list_sell_midprice'], var[stock]['list_sell_midvolume'], var[stock]['list_level_pricefibo'], var[stock]['list_level_volumefibo'], var[stock]['list_avgcost_pricefibo'], var[stock]['list_buy_midcashflow'], var[stock]['list_dif_midcashflow'], var[stock]['list_level_volumefront'])

#--------------------------------------------------------------------------------------------- เริ่มกองหน้า ---------------------------------------------------------------------------------------------#

                def short_trading_front(trade_history_df, var, stock, COMMISSION, VAT):
                    for i, (price_level, volume_level) in enumerate(zip(var[stock]['list_sell_midprice'], var[stock]['list_level_volumefront'])):
                        if var[stock]['count_fibo'] == i + 1 and var[stock]['count_front'] == i and price >= price_level:
                            var[stock]['list_sell_frontprice'].append(price_level)
                            var[stock]['list_sell_frontvolume'].append(volume_level)
                            var[stock]['count_front'] += 1

                            # จัดการกระแสเงินสด
                            keep_cashflow, keep_cash_sell, keep_cash_buy, keep_point_buyback = calculate_front_cashflow(price_level, volume_level, var[stock]['lower_price'], COMMISSION, VAT)
                            update_cashflows(var[stock], keep_cashflow, percentages, 'กองหน้า')
                            var[stock]['buyback_pricefront'] = keep_point_buyback

                            # เก็บประวัติการซื้อขายกองหน้าฝั่งขาย
                            var[stock]['list_data_index_front'].append(var[stock]['data_index'])
                            var[stock]['data_index'] += 1
                            var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                            
                            # อัปเดตประวัติการซื้อขาย
                            new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'FTS', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_frontprice'][-1], var[stock]['list_sell_frontvolume'][-1], 0.00, 0, keep_point_buyback, keep_cash_sell, 0.00, keep_cash_buy, 0.00, 0.00, keep_cashflow, 0.00, 0.00, var[stock]['cashflow_buymore'])
                            df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                short_trading_front(df_trade_hist[stock], var, stock, COMMISSION, VAT)

                # จุดซื้อคืนกองหน้าทั้งหมด
                if  len(var[stock]['list_sell_frontprice']) > 0 and price <= var[stock]['buyback_pricefront']:

                    ## เก็บประวัติการซื้อขายกองหลังฝั่งซื้อ 
                    update_trading_history_front2(df_trade_hist[stock], var[stock]['list_data_index_front'])

                    ## เคลียร์ข้อมูลกองหน้าหลังซื้อคืนได้                 
                    var[stock]['list_sell_frontprice'].clear()
                    var[stock]['list_sell_frontvolume'].clear()
                    var[stock]['list_data_index_front'].clear()
                    var[stock]['buyback_pricefront'] = 0

#--------------------------------------------------------------------------------------------- เริ่มแปลงร่างกองหน้า ---------------------------------------------------------------------------------------------#

                # เช็คว่ามีกองหน้าที่ซื้อคืนยังไม่ได้, ราคาวิ่งไปเกิน 50 ช่อง
                if sum(var[stock]['list_sell_frontvolume']) >= 100 :
                    lower_price_frontconvert = var[stock]['buyback_pricefront']
                    list_namestock_frontconvert = []

                    # หาผลรวมของเงินสดที่ซื้อคืนไม่ได้ของกองหลังทั้งหมดและเพิ่มเข้าไปในหุ้นที่จะแปลงร่าง
                    cashflow_frontconvert = df_trade_hist[stock].loc[(df_trade_hist[stock]['type'] == 'FTS') & (df_trade_hist[stock]['price_buy'] == 0) & (df_trade_hist[stock]['volume_buy'] == 0), 'cash'].sum()

                     # ค้นหาหุ้นที่เข้าเงื่อนไขแปลงร่าง
                    for find_stock in list_stock:
                        
                         # ข้ามหุ้นที่มีชื่อตรงกันกับที่แปลงร่าง
                        if find_stock != stock:   

                            # หาว่าหุ้นตัวไหนว่าราคาปัจจุบันห่างจากต้นทุนแรกมากกว่า 50 ช่อง
                            range_spread_front = calculate_range_of_stock(var[find_stock]['lower_price'], var[find_stock]['current_price'])
                            selected_price_front = calculate_buyback_point(var[find_stock]['current_price'], 5, 1)

                            # เงื่อนไขหุ้นที่รับแปลงร่างมี มีกองหน้า, ราคาต่ำกว่า, อยู่ในช่วง spread เดียวกัน, เงินลงทุนแปลงร่างรวมกับเงินต้นไม่เกิน 10% ของ, เช็คว่าระบบขายหุ้นเพื่อรักษาเงินสดทำงานอยู่
                            if sum(var[find_stock]['list_sell_frontvolume']) >= 100 and selected_price_front < lower_price_frontconvert and calculate_spread(lower_price_frontconvert) == calculate_spread(var[find_stock]['current_price']) and range_spread_front >= 50  and var[find_stock]['count_short'] == 0 and ((cashflow_frontconvert + var[find_stock]['budget']) / TOTAL_BUDGET) <= 0.2 and var[find_stock]['selected_frontcashflow'] == 0: 
                                list_namestock_frontconvert.append({'stock': find_stock, 'range_spread': range_spread_front, 'selected_price': selected_price_front})

                    # หลังจากจบลูป ตรวจสอบว่ามีหุ้นที่ตรงกับเงื่อนไขหรือไม่
                    if list_namestock_frontconvert:
                        
                        # จัดเรียงตาม range_spread จากมากไปน้อย
                        sorted_stocks = sorted(list_namestock_frontconvert, key=lambda x: x['range_spread'], reverse=True)

                        # เลือกหุ้นที่มี range_spread มากที่สุด (อันดับ 1)
                        selected_stock_name = sorted_stocks[0]['stock']   
                        var[selected_stock_name]['selected_frontprice'] = sorted_stocks[0]['selected_price']   
                        var[selected_stock_name]['selected_frontvolume'] = sum(var[stock]['list_sell_frontvolume'])
                        var[selected_stock_name]['selected_frontcashflow'] = cashflow_frontconvert
                        var[selected_stock_name]['selected_frontconvert'] = stock 
                        var[selected_stock_name]['budget'] += cashflow_frontconvert

                        # ปรับเงินลงทุนใหม่
                        var[stock]['budget'] -= cashflow_frontconvert                    

                        # ปรับ update_trading_history_front ใหม่
                        update_trading_frontconvert(var[stock]['list_data_index_front'], df_trade_hist[stock], stock ,var[stock], sorted_stocks[0]['stock'])

                # แปลงร่างเงินสดที่ได้รับมา
                if price <= var[stock]['selected_frontprice'] and var[stock]['selected_frontcashflow'] > 0:

                    net_buy_cashflow = (price * var[stock]['selected_frontvolume']) + (price * var[stock]['selected_frontvolume'] * COMMISSION) + (price * var[stock]['selected_frontvolume'] * COMMISSION * VAT)
                    net_cashflow = var[stock]['selected_frontcashflow'] - net_buy_cashflow
                    var[stock]['sum_volume'] += var[stock]['selected_frontvolume']
                    var[var[stock]['selected_frontconvert']]['sum_volume'] -= var[stock]['selected_frontvolume']

                    #### จัดการกระแสเงินสด
                    update_cashflows(var[stock], net_cashflow, percentages, 'แปลงร่าง')

                    #### เก็บประวัติการซื้อขายกองหลังฝั่งขาย
                    var[stock]['list_data_index_frontconvert'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1 
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'CV receive FTS {}'.format(var[stock]['selected_frontconvert']), var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], 0.00, 0, price, var[stock]['selected_frontvolume'], 0, var[stock]['selected_frontcashflow'], net_buy_cashflow, 0.00, 0.00, 0.00, 0.00, net_cashflow, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)
                    var[stock]['selected_frontcashflow'] = 0


#--------------------------------------------------------------------------------------------- เริ่มแปลงร่างกองหลัง ---------------------------------------------------------------------------------------------#
                    

                # เช็คจำนวนกองหลังว่ามีครบ 3 กองหลังก่อนแปลงร่างและราคาวิ่งไปหาจากกองหลังตัวบนสุดเกิน 50 ช่องหรือไม่          
                if var[stock]['count_back'] == 3 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50 and var[stock]['start_send_convert'] == False :                 
                    lower_price_convert = calculate_buyback_point(var[stock]['lower_price'], 10, -1)
                    list_namestock_convert = []

                    # หาผลรวมของเงินสดที่ซื้อคืนไม่ได้ของกองหลังทั้งหมดและเพิ่มเข้าไปในหุ้นที่จะแปลงร่าง
                    cashflow_convert = df_trade_hist[stock].loc[(df_trade_hist[stock]['type'] == 'BTS') & (df_trade_hist[stock]['price_buy'] == 0) & (df_trade_hist[stock]['volume_buy'] == 0), 'cash'].sum()
                    
                    # หาจำนวนกองหลังที่ซื้อคืนไม่ได้ทั้งหมดและเพิ่มเข้าไปในหุ้นที่จะแปลงร่าง
                    volume_convert = df_trade_hist[stock].loc[(df_trade_hist[stock]['type'] == 'BTS') & (df_trade_hist[stock]['cash'] > 0) & (df_trade_hist[stock]['price_buy'] == 0) & (df_trade_hist[stock]['volume_buy'] == 0), 'volume_sell'].sum()

                    # ค้นหาหุ้นที่เข้าเงื่อนไขแปลงร่าง
                    for find_stock in list_stock:
                        
                         # ข้ามหุ้นที่มีชื่อตรงกันกับที่แปลงร่าง
                        if find_stock != stock:

                            # หาว่าหุ้นตัวไหนว่าราคาปัจจุบันห่างจากต้นทุนแรกมากกว่า 50 ช่อง
                            range_spread = calculate_range_of_stock(var[find_stock]['lower_price'], var[find_stock]['current_price'])
                            selected_price = calculate_buyback_point(var[find_stock]['current_price'], 5, 1)

                            # เงื่อนไขหุ้นที่รับแปลงร่างมี ราคาต่ำกว่า, อยู่ในช่วง spread เดียวกัน, เงินลงทุนแปลงร่างรวมกับเงินต้นไม่เกิน 10% ของ, เช็คว่าระบบขายหุ้นเพื่อรักษาเงินสดทำงานอยู่
                            if selected_price < lower_price_convert and calculate_spread(lower_price_convert) == calculate_spread(var[find_stock]['current_price']) and range_spread >= 50  and var[find_stock]['count_short'] == 0 and ((cashflow_convert + var[find_stock]['budget']) / TOTAL_BUDGET) <= 0.2 and var[find_stock]['selected_cashflow'] == 0  :                           
                                list_namestock_convert.append({'stock': find_stock, 'range_spread': range_spread, 'selected_price': selected_price})


                    # หลังจากจบลูป ตรวจสอบว่ามีหุ้นที่ตรงกับเงื่อนไขหรือไม่
                    if list_namestock_convert:
                        
                        # จัดเรียงตาม range_spread จากมากไปน้อย
                        sorted_stocks = sorted(list_namestock_convert, key=lambda x: x['range_spread'], reverse=True)

                        # เลือกหุ้นที่มี range_spread มากที่สุด (อันดับ 1)
                        selected_stock_name = sorted_stocks[0]['stock']   
                        var[selected_stock_name]['selected_price'] = sorted_stocks[0]['selected_price']   
                        var[selected_stock_name]['volume_convert'] = volume_convert
                        var[selected_stock_name]['selected_cashflow'] = cashflow_convert
                        var[selected_stock_name]['selected_convert'] = stock  
                        var[selected_stock_name]['start_recieve_convert'] = True
                        var[selected_stock_name]['budget'] += cashflow_convert

                        # ปรับเงินลงทุนใหม่
                        var[stock]['budget'] -= cashflow_convert                    
                        var[stock]['start_send_convert'] = True

                        # ปรับ update_trading_history_back ใหม่
                        update_trading_backconvert(var[stock]['list_data_index_back'], df_trade_hist[stock], stock ,var[stock], sorted_stocks[0]['stock'])
                        

                # แปลงร่างเงินสดที่ได้รับมา
                if  var[stock]['start_recieve_convert'] and price <= var[stock]['selected_price'] and var[stock]['selected_cashflow'] > 0:
                    
                    net_buy_cashflow = (price * var[stock]['volume_convert']) + (price * var[stock]['volume_convert'] * COMMISSION) + (price * var[stock]['volume_convert'] * COMMISSION * VAT)
                    net_cashflow = var[stock]['selected_cashflow'] - net_buy_cashflow
                    var[stock]['sum_volume'] += var[stock]['volume_convert']
                    var[stock]['start_recieve_convert'] = False
                    var[var[stock]['selected_convert']]['sum_volume'] -= var[stock]['volume_convert']

                    #### จัดการกระแสเงินสด
                    update_cashflows(var[stock], net_cashflow, percentages, 'แปลงร่าง')

                    #### เก็บประวัติการซื้อขายกองหลังฝั่งขาย
                    var[stock]['list_data_index_convert'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1 
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'CV receive BTS {}'.format(var[stock]['selected_convert']), var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], 0.00, 0, price, var[stock]['volume_convert'], 0, var[stock]['selected_cashflow'], net_buy_cashflow, 0.00, 0.00, 0.00, 0.00, net_cashflow, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)
                    var[stock]['selected_cashflow'] = 0
               

#--------------------------------------------------------------------------------------------- เริ่มแปลงร่าง sl ---------------------------------------------------------------------------------------------#

                       
                if var[stock]['count_slzone'] == 1:
                    var[stock]['list_namestock_slconvert'].clear()

                    # ค้นหาหุ้นที่เข้าเงื่อนไขแปลงร่าง
                    for find_stock in list_stock:
                        
                         # ข้ามหุ้นที่มีชื่อตรงกันกับที่แปลงร่าง
                        if find_stock != stock:

                            # หาว่าหุ้นตัวไหนว่าราคาปัจจุบันห่างจากต้นทุนแรกมากกว่า 50 ช่อง
                            range_spread = calculate_range_of_stock(var[find_stock]['lower_price'], var[find_stock]['current_price'])
                            selected_price = calculate_buyback_point(var[find_stock]['current_price'], 5, 1)

                            # เงื่อนไขหุ้นที่รับแปลงร่างมี ราคาต่ำกว่า, อยู่ในช่วง spread เดียวกัน, เงินลงทุนแปลงร่างรวมกับเงินต้นไม่เกิน 10% ของ, เช็คว่าระบบขายหุ้นเพื่อรักษาเงินสดทำงานอยู่, ราคาต้องอยู่ต่ำกว่าจุดซื้อคืนต่ำสุด
                            if selected_price < var[stock]['lower_price_convert'] and calculate_spread(var[stock]['lower_price_convert']) == calculate_spread(var[find_stock]['current_price']) and range_spread >= 50  and ((var[stock]['cashflow_slconvert'] + var[find_stock]['budget']) / TOTAL_BUDGET) <= 0.2  and var[stock]['volume_slconvert'] > 0 and var[find_stock]['selected_slcashflow'] == 0:                           
                                var[stock]['list_namestock_slconvert'].append({'stock': find_stock, 'range_spread': range_spread, 'selected_price': selected_price})
                                var[stock]['count_slzone'] = 2


                    # หลังจากจบลูป ตรวจสอบว่ามีหุ้นที่ตรงกับเงื่อนไขหรือไม่
                    if var[stock]['list_namestock_slconvert'] and var[stock]['count_slzone'] == 2:
                    
                        # จัดเรียงตาม range_spread จากมากไปน้อย
                        sorted_stocks = sorted(var[stock]['list_namestock_slconvert'], key=lambda x: x['range_spread'], reverse=True)

                        # เลือกหุ้นที่มี range_spread มากที่สุด (อันดับ 1)
                        selected_stock_name = sorted_stocks[0]['stock']   
                        var[selected_stock_name]['selected_slprice'] = sorted_stocks[0]['selected_price']   
                        var[selected_stock_name]['selected_slvolume'] = var[stock]['volume_slconvert']
                        var[selected_stock_name]['selected_slcashflow'] = var[stock]['cashflow_slconvert']
                        var[selected_stock_name]['selected_slconvert'] = stock 
                        var[selected_stock_name]['budget'] += var[stock]['cashflow_slconvert']

                        # ปรับ update_trading_history_back ใหม่
                        for index in var[stock]['list_data_index_sl']: 
                            df_trade_hist[stock].loc[index, 'cash'] = 0.00
                            df_trade_hist[stock].loc[index, 'type'] = 'SL CV send {}'.format(selected_stock_name)
                        var[stock]['list_data_index_sl'].clear() 

                   
                       
                # แปลงร่างเงินสดที่ได้รับมา
                if  price <= var[stock]['selected_slprice'] and var[stock]['selected_slcashflow'] > 0:
                    
                    selected_slcashflow = var[stock]['selected_slcashflow']

                    def calculate_net_cashflow(volume, price, commission, vat):
                        net_buy_cashflow = (price * volume) + (price * volume * commission) + (price * volume * commission * vat)
                        return net_buy_cashflow                  

                    # เริ่มต้นด้วย net_buy_volume ที่ 74300 จากการคำนวณ
                    net_buy_volume = calculate_floor_stock(selected_slcashflow / price)
                    net_cashflow = -1  # เริ่มต้นด้วยค่าติดลบเพื่อเข้า loop

                    while net_cashflow < 0 and net_buy_volume >= 100:
                        net_buy_cashflow = calculate_net_cashflow(net_buy_volume, price, COMMISSION, VAT)
                        net_cashflow = selected_slcashflow - net_buy_cashflow
                        if net_cashflow < 0:
                            # ลด net_buy_volume ลงทีละ 100 หุ้น
                            net_buy_volume -= 100

                    
                    #### จัดการกระแสเงินสด
                    update_cashflows(var[stock], net_cashflow, percentages, 'แปลงร่าง')
                    var[stock]['sum_volume'] += net_buy_volume
                    var[var[stock]['selected_slconvert']]['sum_volume'] -= var[stock]['selected_slvolume']

                    #### เก็บประวัติการซื้อขายกองหลังฝั่งขาย
                    var[stock]['list_data_index_sl'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1 
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'SL CV receive {}'.format(var[stock]['selected_slconvert'] ), var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], 0.00, 0, price, net_buy_volume, 0, var[stock]['selected_slcashflow'], net_buy_cashflow, 0.00, 0.00, 0.00, 0.00, net_cashflow, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)
                    var[stock]['selected_slcashflow'] = 0

                # ราคาเพิ่มมากขึ้นกว่า 50 ช่องจากจุดต่ำสุดเริ่มลงทุนใหม่อีกครั้ง
                if var[stock]['count_slzone'] == 2 and var[stock]['count_sl_convert'] >= 25 and calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price']) >= 50 :
                    var[stock]['count_slzone'] = 0
                    var[stock]['count_sl_convert'] = 0

#--------------------------------------------------------------------------------------------- เริ่มระบบ short ---------------------------------------------------------------------------------------------#

                # หาจุดขายระบบ short 
                if  price <= var[stock]['signal_shortprice'] and var[stock]['count_short'] == 0 and var[stock]['start_in_shortzone'] == False  :

                    # หาจำนวนหุ้นทั้งหมดที่เหลือตอนนั้น ไม่รวมกองกลางเพราะถ้าราคาลงมาถึง fibo level 0 ยังไงก็ซื้อคืนได้หมดแล้ว 
                    net_sell_volume = var[stock]['sum_volume'] - sum(var[stock]['list_sell_backvolume']) - sum(var[stock]['list_sell_frontvolume']) - sum(var[stock]['list_sell_shortvolume'])
                    var[stock]['list_sell_shortprice'].append(var[stock]['signal_shortprice'])
                    var[stock]['list_sell_shortvolume'].append(net_sell_volume)
                    var[stock]['count_short'] = 1
                    var[stock]['start_in_shortzone'] = True


                    # เก็บประวัติระบบเก็บเงินสด
                    find_sell_cash = (var[stock]['list_sell_shortprice'][0] * var[stock]['list_sell_shortvolume'][0]) - (var[stock]['list_sell_shortprice'][0] * var[stock]['list_sell_shortvolume'][0] * COMMISSION) -  (var[stock]['list_sell_shortprice'][0] * var[stock]['list_sell_shortvolume'][0] * COMMISSION * VAT)
                    var[stock]['list_data_index_short'].append(var[stock]['data_index'])
                    var[stock]['data_index'] += 1
                    var[stock]['net_volume_percentage'] ,var[stock]['net_volume'] = calculate_net_volume(var[stock]['list_sell_backvolume'], var[stock]['list_sell_midvolume'], var[stock]['list_sell_frontvolume'], var[stock]['list_sell_shortvolume'], var[stock]['sum_volume'])                        
                    new_trade_hist = update_trade_hist(df[stock].loc[var[stock]['count_index'], 'date'], df[stock].loc[var[stock]['count_index'], 'time'], 'STS', var[stock]['sum_volume'], var[stock]['increase_volume'], var[stock]['net_volume'], var[stock]['avg_price'], var[stock]['list_sell_shortprice'][0] , var[stock]['list_sell_shortvolume'][0], 0.00, 0, 0.00, find_sell_cash, 0.00, find_sell_cash, 0.00, 0.00, 0.00, 0.00, 0.00, var[stock]['cashflow_buymore'])
                    df_trade_hist[stock] = pd.concat([df_trade_hist[stock].astype(new_trade_hist.dtypes), new_trade_hist.astype(df_trade_hist[stock].dtypes)], ignore_index=True)

                # ปรับจุดต่ำสุดใหม่
                if var[stock]['count_short'] >= 1 and var[stock]['start_in_shortzone']:
                    var[stock]['signal_shortprice'] = min(var[stock]['signal_shortprice'], price )


                    # หาสัญญาณเกิดจุดซื้อคืนราคาลงไปต่ำกว่า 5 ช่องจากจุดขาย, หาสัญญาณเกิดจุดซื้อคืนเกิดกองกลาง, หาสัญาณเกิดจุดซื้อคืนต่ำกว่าจุดต่ำสุด                
                    if var[stock]['count_short'] == 1 and price <= calculate_buyback_point(var[stock]['list_sell_shortprice'][0], 5, -1):
                        var[stock]['count_short'] = 2
                    elif var[stock]['count_short'] == 2 and price >= calculate_buyback_point(var[stock]['list_sell_shortprice'][0], 2, -1) :
                        var[stock]['list_buy_shortprice'].append(calculate_buyback_point(var[stock]['list_sell_shortprice'][0], 2, -1))
                        var[stock]['count_short'] = 0

                        ## จัดการกระแสเงินสด
                        keep_cashflow, keep_sell_cashflow, keep_buy_cashflow = calculate_short_cashflow(var[stock]['list_sell_shortprice'], var[stock]['list_sell_shortvolume'], var[stock]['list_buy_shortprice'], COMMISSION, VAT)
                        update_cashflows(var[stock], keep_cashflow, percentages, 'short')
                        update_short_trade_history(df_trade_hist, var, stock, keep_buy_cashflow, keep_cashflow)


                # จุดซื้อคืนเมื่อราคาเกิดกองกลาง
                if  var[stock]['start_in_shortzone'] and price >= df[stock]['up_band'].iloc[var[stock]['count_index']] and price <= var[stock]['list_sell_shortprice'][0] :
                    var[stock]['list_buy_shortprice'].append(price)
                    var[stock]['signal_shortprice'] = 0
                    var[stock]['count_short'] = 0

                    ## จัดการกระแสเงินสด
                    keep_cashflow, keep_sell_cashflow, keep_buy_cashflow = calculate_short_cashflow(var[stock]['list_sell_shortprice'], var[stock]['list_sell_shortvolume'], var[stock]['list_buy_shortprice'], COMMISSION, VAT)
                    update_cashflows(var[stock], keep_cashflow, percentages, 'short')
                    update_short_trade_history(df_trade_hist, var, stock, keep_buy_cashflow, keep_cashflow)

                # จุดซื้อคืนเมื่อราคาลงมาถึงกองหลังตัวบนสุดและถ้าไม่มีกองหลังใช้ราคาต้นทุน
                if  var[stock]['start_in_shortzone'] and price <= var[stock]['lower_price'] and price <= var[stock]['list_sell_shortprice'][0] :
                    var[stock]['list_buy_shortprice'].append(var[stock]['lower_price'])
                    var[stock]['signal_shortprice'] = 0
                    var[stock]['count_short'] = 0
                    
                    ## จัดการกระแสเงินสด
                    keep_cashflow, keep_sell_cashflow, keep_buy_cashflow = calculate_short_cashflow(var[stock]['list_sell_shortprice'], var[stock]['list_sell_shortvolume'], var[stock]['list_buy_shortprice'], COMMISSION, VAT)
                    update_cashflows(var[stock], keep_cashflow, percentages, 'short')
                    update_short_trade_history(df_trade_hist, var, stock, keep_buy_cashflow, keep_cashflow)


                ## เก็บประวัติการซื้อขาย short
                def update_short_trade_history(df_trade_hist, var, stock, keep_buy_cashflow, keep_cashflow):
                    # ตรวจสอบว่ามีข้อมูลการซื้อขาย short หรือไม่
                    index = var[stock]['list_data_index_short'][0] 
                    df_trade_hist[stock].loc[index, 'price_buy'] = var[stock]['list_buy_shortprice'][0]
                    df_trade_hist[stock].loc[index, 'volume_buy'] = df_trade_hist[stock].loc[index, 'volume_sell']
                    df_trade_hist[stock].loc[index, 'cash_buy'] = keep_buy_cashflow
                    df_trade_hist[stock].loc[index, 'cashflow_short'] = keep_cashflow
                    df_trade_hist[stock].loc[index, 'cash'] = 0.00

                    # ล้างลิสต์ที่เกี่ยวข้องหลังจากอัพเดตประวัติ
                    var[stock]['start_in_shortzone'] = False
                    var[stock]['list_sell_shortprice'].clear()
                    var[stock]['list_sell_shortvolume'].clear()
                    var[stock]['list_buy_shortprice'].clear()
                    var[stock]['list_data_index_short'].clear()


#--------------------------------------------------------------------------------------------- บันทึกข้อมูลรายวัน ---------------------------------------------------------------------------------------------#

                var[stock]['total_equity'] = price * (var[stock]['sum_volume'] - (sum(var[stock]['list_sell_backvolume']) + sum(var[stock]['list_sell_midvolume']) + sum(var[stock]['list_sell_frontvolume']) + sum(var[stock]['list_sell_shortvolume'])))
                var[stock]['benchmark'] = price * var[stock]['FIRST_VOLUME']
                var[stock]['date'] = df[stock].loc[var[stock]['count_index'], 'date']



#--------------------------------------------------------------------------------------------- จบแท่ง 1 ชั่วโมง ---------------------------------------------------------------------------------------------#
         
            # บวก index เพิ่มตาม
            var[stock]['count_index'] += 1

#--------------------------------------------------------------------------------------------- จัดอันดับหุ้นที่วิ่งไปได้ไกลที่สุด 5 อันดับแรก ---------------------------------------------------------------------------------------------#

    # สร้าง list ของ dictionaries ที่มีชื่อหุ้นและระยะห่างของหุ้นนั้น
    spread_list = []
    for stock in list_stock:
        spread = calculate_range_of_stock(var[stock]['lower_price'], var[stock]['current_price'])
        spread_list.append({'stock': stock, 'range_spread': spread})

    # จัดเรียง spread_list ตาม 'range_spread' จากมากไปหาน้อย
    sorted_list_stock = sorted(spread_list, key=lambda x: x['range_spread'], reverse=True)

    # เลือก 5 อันดับแรก
    top_5_stock = sorted_list_stock[:5]

    # ตรวจสอบว่าหุ้นใดๆ ใน top 5 มี spread >= 50
    if any(item['range_spread'] >= 50 for item in top_5_stock):
        # คำนวณผลรวมของ spread ใน top 5
        total_spread = sum(item['range_spread'] for item in top_5_stock if item['range_spread'] >= 50)
    else:
        total_spread = 0

    # คำนวณสัดส่วนเป็นเปอร์เซ็นต์และอัปเดตใน top_5_stock
    if total_spread > 0:
        for item in top_5_stock:
            if item['range_spread'] >= 50:
                item['spread_percentage'] = round((item['range_spread'] / total_spread), 4)
            else:
                item['spread_percentage'] = 0

        # จัดสรรเงินทุนตามสัดส่วน spread_percentage
        for item in top_5_stock:
            if item['spread_percentage'] > 0:
                stock = item['stock']
                allocated_funds = dict_all_sum_port['cashflow_buymore'] * item['spread_percentage']
                var[stock]['cashflow_buymore'] += allocated_funds
        dict_all_sum_port['cashflow_buymore'] = 0
    else:
        for item in top_5_stock:
            item['spread_percentage'] = 0


#--------------------------------------------------------------------------------------------- พอร์ตรวมข้อมูลของหุ้นทุกตัวรายวัน ---------------------------------------------------------------------------------------------#
    # โหลดข้อมูลย้อนหลังของหุ้นแต่ละตัวในวันที่กำหนด
    for stock in list_stock :  
        dict_keep_variable['cashflow_reserve'] = dict_all_sum_port['cashflow_reserve']
        dict_keep_variable['cashflow_fee'] = dict_all_sum_port['cashflow_fee']
        dict_keep_variable['sum_cashflow_back'] = dict_all_sum_port['sum_cashflow_back']
        dict_keep_variable['sum_cashflow_mid'] = dict_all_sum_port['sum_cashflow_mid']
        dict_keep_variable['sum_cashflow_front'] = dict_all_sum_port['sum_cashflow_front']
        dict_keep_variable['sum_cashflow_convert'] = dict_all_sum_port['sum_cashflow_convert']
        dict_keep_variable['sum_cashflow_short'] = dict_all_sum_port['sum_cashflow_short']


        # เริ่มเก็บประวัติ
        if var[stock]['start_in_tradezone']:  
            dict_keep_variable['sum_volume'] += var[stock]['sum_volume']
            dict_keep_variable['increase_volume'] += var[stock]['increase_volume']
            dict_keep_variable['net_volume'] += var[stock]['sum_volume'] - (sum(var[stock]['list_sell_backvolume']) + sum(var[stock]['list_sell_midvolume']) + sum(var[stock]['list_sell_frontvolume']) + sum(var[stock]['list_sell_shortvolume']))
            dict_keep_variable['cash_buyback'] += df_trade_hist[stock]['cash'].sum(axis=0)
            dict_keep_variable['total_cash'] += df_trade_hist[stock]['cash'].sum(axis=0) + var[stock]['cashflow_buymore']
            dict_keep_variable['total_equity'] += var[stock]['total_equity'] 
            dict_keep_variable['benchmark'] += var[stock]['benchmark']

            # นำเงินซื้อเพิ่มของหุ้นทุกตัวที่เหลืออยู่มารวมด้วย
            dict_keep_variable['cashflow_buymore'] += var[stock]['cashflow_buymore']

    # เก็บข้อมูลทั้งหมดลง dataframe รวมพอร์ต
    if var[stock]['start_in_tradezone']: 
        dict_keep_variable['cashflow_buymore'] += dict_all_sum_port['cashflow_buymore']
        dict_keep_variable['total_cash'] += dict_all_sum_port['cashflow_buymore'] + dict_all_sum_port['cashflow_reserve'] +  dict_all_sum_port['cashflow_fee']
        dict_keep_variable['nav'] = dict_keep_variable['total_equity'] + dict_keep_variable['total_cash']
    new_dt = update_sum_port(var[stock]['date'], dict_keep_variable['sum_volume'], dict_keep_variable['increase_volume'], dict_keep_variable['net_volume'], dict_keep_variable['sum_cashflow_back'], dict_keep_variable['sum_cashflow_mid'], dict_keep_variable['sum_cashflow_front'], dict_keep_variable['sum_cashflow_convert'], dict_keep_variable['sum_cashflow_short'], dict_keep_variable['cash_buyback'], dict_keep_variable['cashflow_buymore'], dict_keep_variable['cashflow_reserve'],  dict_keep_variable['cashflow_fee'], dict_keep_variable['total_cash'], dict_keep_variable['total_equity'], dict_keep_variable['nav'] , dict_keep_variable['benchmark'])
    
    # แปลงคอลัมน์ 'date' ใน new_dt ไปเป็น datetime และจัดการกับข้อผิดพลาด
    new_dt['date'] = pd.to_datetime(new_dt['date'], errors='coerce')
    # ลบคอลัมน์ที่มีค่า NA ทั้งหมดจาก new_dt
    new_dt = new_dt.dropna(how='all', axis=1)
    df_dt = df_dt.dropna(how='all', axis=1)
    # ทำการ concat
    df_dt = pd.concat([df_dt, new_dt], ignore_index=True)


#--------------------------------------------------------------------------------------------- แปลงข้อมูล df ให้อยู่ในรูปแบบทศนิยม 2 ตำแหน่ง ---------------------------------------------------------------------------------------------#
            
    columns_to_round = ['sum_volume' ,'increase_volume' , 'price_sell', 'price_buy', 'point_buyback', 'cash_sell', 'cash_buy', 'cash', 'cashflow_back', 'cashflow_mid', 'cashflow_front', 'cashflow_short']
    df_trade_hist[stock][columns_to_round] = df_trade_hist[stock][columns_to_round].round(2)

    columns_to_round3 = [ 'cash_buyback', 'cashflow_buymore', 'cashflow_reserve', 'cashflow_back', 'cashflow_mid', 'cashflow_front', 'cashflow_convert', 'cashflow_short', 'cashflow_fee', 'total_cash', 'total_equity','nav','benchmark']
    df_dt[columns_to_round3] = df_dt[columns_to_round3].round(2)


In [2424]:
len(list_stock)

47

In [2416]:
TOTAL_BUDGET

94000000

In [2417]:
# 1. ตรวจสอบและแปลงคอลัมน์ 'date' เป็นชนิดข้อมูล datetime
df_dt['date'] = pd.to_datetime(df_dt['date'])

# 2. สร้างคอลัมน์ใหม่ 'end_of_month' เพื่อเก็บวันที่สุดท้ายของเดือน
df_dt['end_of_month'] = df_dt['date'].dt.to_period('M').dt.to_timestamp('M')

# 3. กรองข้อมูลเพื่อเลือกเฉพาะแถวที่วันที่ตรงกับวันสุดท้ายของเดือน
last_day_of_month_df = df_dt.groupby(df_dt['date'].dt.to_period('M')).apply(lambda x: x.iloc[-1])

# ลบคอลัมน์ 'end_of_month' และ 'benchmark'
last_day_of_month_df = last_day_of_month_df.drop(columns=['end_of_month'])
df_dt = df_dt.drop(columns=['end_of_month'])

last_day_of_month_df

,sum_volume,increase_volume,net_volume,cashflow_back,cashflow_mid,cashflow_front,cashflow_convert,cashflow_short,cash_buyback,cashflow_buymore,cashflow_reserve,cashflow_fee,total_cash,total_equity,nav,benchmark,date
date,,,,,,,,,,,,,,,,,
2022-02,5410800,0,5127900,216479.50,13820.60,26626.37,0.00,0.00,7320270.14,179848.53,51385.29,25692.65,7577196.61,83402048.0,9.097924e+07,90990213.0,2022-02-28
2022-03,5483400,0,4102300,1125179.30,47050.12,132261.02,0.00,0.00,20554220.97,913143.31,260898.09,130449.04,21858711.41,73092832.0,9.495154e+07,95718607.0,2022-03-31
2022-04,5483400,0,4007600,1306654.13,71251.60,186027.80,100.11,13649.88,25578147.51,1104378.47,315536.70,157768.35,27155831.03,66157896.0,9.331373e+07,93611908.0,2022-04-29
2022-05,5483400,0,3926600,1716573.14,91948.49,247334.98,100.11,105277.12,25034389.21,1512863.69,432246.77,216123.38,27195623.05,67765404.0,9.496103e+07,94683373.0,2022-05-31
2022-06,5484300,900,3816700,1995023.48,111027.58,304122.77,3103.55,225787.31,29859480.35,1691018.17,527812.94,263906.47,32342217.93,61056374.0,9.339859e+07,90946327.0,2022-06-30
2022-07,5485900,2500,3722100,2243213.98,133798.53,383190.67,4505.15,378199.41,33694060.43,1756980.41,628581.55,314290.77,36393913.16,60346346.0,9.674026e+07,92577036.0,2022-07-27
2022-08,5491100,2900,3588700,2251462.28,198718.96,601927.38,55008.40,387070.68,35500647.70,1929588.24,698837.54,349418.77,38478492.24,58403822.0,9.688231e+07,96189609.0,2022-08-31
2022-09,5504300,6000,3578600,2608317.36,229275.08,870730.65,146535.37,513380.54,36560596.87,2372703.04,873647.80,436823.90,40243771.61,59216093.0,9.945986e+07,93093301.0,2022-09-30
2022-10,5508000,9100,3411800,2859894.06,245255.00,1013482.95,154517.77,590856.68,41107698.14,2518682.74,972801.29,486400.65,45085582.82,53836158.0,9.892174e+07,94903272.0,2022-10-31


# บันทึกข้อมูลเป็น Excel

In [2418]:
output_path_history_trading = r'C:\Users\ming\Desktop\MyJob\DSM1.1.2\history\history_trading.xlsx'
output_path_port_trading = r'C:\Users\ming\Desktop\MyJob\DSM1.1.2\history\port_trading.xlsx'
output_path_history_trading = r'C:\Users\ming\Desktop\MyJob\DSM1.1.2\history\history_trading.xlsx'


# แปลงข้อมูลประวัติรายวันเป็น excel
df_dt.to_excel(output_path_port_trading, index=False)  # ใช้ `index=False` เพื่อไม่บันทึกดัชนี DataFrame

# แปลงข้อมูลประวัติรายวันเป็น excel
last_day_of_month_df.to_excel(output_path_month_trading, index=False)  # ใช้ `index=False` เพื่อไม่บันทึกดัชนี DataFrame

# แปลงข้อมูลประวัติการซื้อขายเป็น excel
with pd.ExcelWriter(output_path_history_trading) as writer:
    for stock in list_stock:
        # เขียน DataFrame to a separate sheet
        df_trade_hist[stock].to_excel(writer, sheet_name=stock)

In [2419]:
dict_all_sum_port['cashflow_buymore'] + dict_all_sum_port['cashflow_reserve'] + dict_all_sum_port['cashflow_fee'] 

4727080.468933405

In [2420]:
dict_all_sum_port['sum_cashflow_back'] 

4628262.069999999

In [2421]:
dict_all_sum_port['sum_cashflow_mid'] 

660080.5599999995

In [2422]:
dict_all_sum_port['sum_cashflow_front'] 

5721110.969999999

In [2423]:
for stock in list_stock:
    print(stock, var[stock]['sum_volume'])

SCC 1500
ADVANC 0
KBANK 3600
KCE 34600
EA -100
CBG 100
DELTA 183600
INTUCH 25200
BH 21000
BBL 12200
SCGP 9500
SAWAD 34000
KTC 9000
PTTEP 21100
GULF 39800
TOP 39800
EGCO 3300
MTC 38600
GPSC 0
TISCO 12600
BGRIM 17700
PTTGC 34900
AOT 19700
CPALL 29900
CPN 43000
RATCH 44400
TU 95200
IVL 40800
CENTEL 99600
OR 23700
CRC 64100
MINT 62800
PTT 50600
CPF 23500
OSP 18300
BANPU 54000
GLOBAL 128700
BDMS 119000
KTB 360200
True 549700
HMPRO 139800
BEM 230100
AWC 355900
LH 201900
BTS 64500
WHA 787100
TTB 1459800
